# Multi-Agent Medical System Orchestration Demonstration

## Overview

This notebook demonstrates the **Agent Orchestration System** of our Agentic Medical Intelligence System, replicating the behavior of `agent_decision.py` using **LangGraph** for intelligent agent routing and workflow management. This system represents the core decision-making engine that coordinates between specialized medical AI agents to provide comprehensive healthcare assistance.

### LangGraph Agent Orchestration Architecture:

Our system employs a sophisticated multi-agent architecture powered by LangGraph, featuring:

1. **CONVERSATION_AGENT**: General medical consultation and patient interaction
2. **RAG_AGENT**: Medical literature retrieval from pre-ingested knowledge base
3. **WEB_SEARCH_PROCESSOR_AGENT**: Current medical information and recent developments
4. **BRAIN_TUMOR_AGENT**: YOLO-based brain MRI tumor detection and visualization
5. **CHEST_XRAY_AGENT**: DenseNet COVID-19 detection in chest X-rays
6. **SKIN_LESION_AGENT**: U-Net skin lesion segmentation and analysis

### Key Orchestration Features:

- **Intelligent Routing**: Automatic agent selection based on query content and context
- **Direct Image Routing**: Bypass expensive LLM calls for classified medical images
- **Confidence-Based Fallbacks**: RAG → Web Search routing when local knowledge insufficient
- **Human Validation Workflows**: Medical diagnosis validation and oversight
- **Guardrails Integration**: Input/output safety and ethical content filtering
- **Conversation Memory**: Persistent state management across interactions

### Academic Significance:

This demonstration showcases advanced concepts in:
- **Multi-Agent Systems**: Coordination and communication between specialized AI agents
- **Workflow Orchestration**: LangGraph state management and conditional routing
- **Medical AI Integration**: Combining local models with cloud-based reasoning
- **Decision Tree Logic**: Transparent, explainable agent selection algorithms
- **Human-AI Collaboration**: Validation workflows for critical medical decisions

---

**Ethics Statement**: This system is designed for educational and research purposes. All medical decisions require validation by qualified healthcare professionals.

## 1. Environment Setup and Dependencies

This section initializes the comprehensive environment required for multi-agent orchestration. Our technical stack includes:

### Core Orchestration Frameworks:
- **LangGraph**: Advanced workflow orchestration and state management
- **LangChain**: LLM integration and prompt engineering
- **Qdrant**: Vector database for medical literature retrieval
- **OpenAI GPT**: Decision-making and natural language processing

### Medical AI Integration:
- **Custom Agent Classes**: Specialized medical AI model wrappers
- **Guardrails System**: Safety and ethical content validation
- **Configuration Management**: Centralized system settings and parameters

### Visualization and Monitoring:
- **Matplotlib**: Agent decision flow visualization
- **NetworkX**: LangGraph structure representation
- **Logging**: Comprehensive system monitoring and debugging

In [1]:
# Import essential libraries for multi-agent orchestration
import os
import sys
import json
import time
import logging
import warnings
from typing import Dict, List, Optional, Any, Literal, TypedDict, Union, Annotated
from pathlib import Path

# Suppress warnings for cleaner academic presentation
warnings.filterwarnings('ignore')

# LangGraph and LangChain components for agent orchestration
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough
from langgraph.graph import MessagesState, StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Environment and configuration management
from dotenv import load_dotenv
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from datetime import datetime

# Add project root to Python path for agent imports
sys.path.append('/Users/sourangshupal/Downloads/Agentic-Medical-Intelligence-System')

# Load environment variables containing API keys
load_dotenv()

# Configure logging for agent orchestration monitoring
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(),  # Console output
        logging.FileHandler('agent_orchestration.log', mode='a')  # File logging
    ]
)
logger = logging.getLogger('AgentOrchestration')

# Configure matplotlib for high-quality academic visualizations
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 10

print("✅ Multi-agent orchestration environment initialized successfully")
print(f"📍 Working directory: {os.getcwd()}")
print(f"🐍 Python version: {sys.version.split()[0]}")
print(f"⏰ Session started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📝 Logging enabled: agent_orchestration.log")

✅ Multi-agent orchestration environment initialized successfully
📍 Working directory: /Users/sourangshupal/Downloads/Agentic-Medical-Intelligence-System
🐍 Python version: 3.11.6
⏰ Session started: 2025-08-11 14:57:20
📝 Logging enabled: agent_orchestration.log


In [2]:
# Import specialized medical agent components
# These represent the core medical AI capabilities of our system

try:
    from config import Config
    from agents.rag_agent import MedicalRAG
    from agents.web_search_processor_agent import WebSearchProcessorAgent
    from agents.image_analysis_agent import ImageAnalysisAgent
    from agents.guardrails.local_guardrails import LocalGuardrails
    
    # Initialize system configuration with all agent settings
    config = Config()
    
    print("🔬 Medical agent orchestration components imported successfully")
    print("\n📋 System Configuration Loaded:")
    print(f"   🧠 Decision Model: {config.agent_decision.llm}")
    print(f"   👁️ Vision Model: GPT-4o Vision for image classification")
    print(f"   📚 RAG Collection: {config.rag.collection_name}")
    print(f"   🔍 Web Search: Tavily API integration")
    print(f"   🛡️ Guardrails: Local safety validation enabled")
    print(f"   💾 Memory: LangGraph conversation persistence")
    
except ImportError as e:
    print(f"❌ Error importing agent components: {e}")
    print("Please ensure all medical agent modules are available")
    raise
except Exception as e:
    print(f"❌ Configuration error: {e}")
    raise

2025-08-11 14:57:25,280 - agents.image_analysis_agent.brain_tumor_agent.brain_tumor_inference - INFO - Using device: cpu
2025-08-11 14:57:25,282 - agents.image_analysis_agent.skin_lesion_agent.skin_lesion_inference - INFO - Using device: cpu


🔬 Medical agent orchestration components imported successfully

📋 System Configuration Loaded:
   🧠 Decision Model: client=<openai.resources.chat.completions.completions.Completions object at 0x334b63990> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x33555a050> root_client=<openai.OpenAI object at 0x334b6f910> root_async_client=<openai.AsyncOpenAI object at 0x333ff9050> model_name='gpt-4' temperature=0.1 model_kwargs={} openai_api_key=SecretStr('**********')
   👁️ Vision Model: GPT-4o Vision for image classification
   📚 RAG Collection: medirag
   🔍 Web Search: Tavily API integration
   🛡️ Guardrails: Local safety validation enabled
   💾 Memory: LangGraph conversation persistence


## 2. Agent State and Configuration Classes

This section defines the foundational data structures for our multi-agent system. The design follows enterprise-grade software architecture principles:

### AgentState Class:
Extends LangGraph's `MessagesState` to track:
- **Conversation History**: Complete message chain with context preservation
- **Agent Metadata**: Current active agent and routing decisions
- **Input Processing**: Text, image, and multi-modal input handling
- **Validation Flags**: Human oversight requirements for medical decisions
- **Confidence Metrics**: Quantitative assessment of response reliability

### AgentConfig Class:
Centralizes system parameters:
- **Model Selection**: GPT-4o for decision-making and vision analysis
- **Confidence Thresholds**: Evidence-based routing criteria
- **System Prompts**: Carefully engineered instructions for medical triage
- **Agent Routing Logic**: Transparent decision-making algorithms

### Clinical Decision-Making Algorithms:
Our routing system implements clinical triage principles:
1. **Image Priority**: Medical images receive immediate specialized routing
2. **Knowledge Hierarchy**: Local literature → Web search → Human validation
3. **Safety First**: All medical diagnoses require human oversight
4. **Confidence Thresholds**: Statistical validation of response reliability

In [3]:
# Define the comprehensive agent state structure for multi-agent coordination
# This extends LangGraph's MessagesState with specialized medical workflow requirements

class AgentState(MessagesState):
    """
    Extended state management for medical multi-agent system.
    
    This class maintains comprehensive state information across the agent workflow,
    including conversation history, agent metadata, validation requirements,
    and confidence metrics essential for medical decision-making.
    
    Attributes:
        messages: List[BaseMessage] - Complete conversation history (inherited)
        agent_name: Current active agent identifier
        current_input: Raw user input (text, dict with image, or multi-modal)
        has_image: Boolean flag for image presence detection
        image_type: Medical image classification (MRI, X-ray, dermoscopy)
        output: Final response content for user delivery
        needs_human_validation: Medical oversight requirement flag
        retrieval_confidence: RAG system confidence score (0.0-1.0)
        bypass_routing: Guardrails bypass flag for safety violations
        insufficient_info: RAG knowledge gap indicator for web search routing
    """
    # Core agent workflow state
    agent_name: Optional[str]  # Current processing agent identifier
    current_input: Optional[Union[str, Dict]]  # User input to be processed
    
    # Image processing state
    has_image: bool  # Flag indicating image upload presence
    image_type: Optional[str]  # Classified medical image type for routing
    
    # Response and validation state
    output: Optional[str]  # Generated response for user delivery
    needs_human_validation: bool  # Medical diagnosis validation requirement
    
    # Quality and routing metrics
    retrieval_confidence: float  # RAG retrieval confidence score
    bypass_routing: bool  # Safety guardrails bypass flag
    insufficient_info: bool  # Knowledge gap indicator for routing decisions


class AgentDecision(TypedDict):
    """
    Structured output format for agent routing decisions.
    
    This standardized format ensures consistent decision-making across
    all routing scenarios, providing transparency and auditability
    for clinical decision support systems.
    
    Attributes:
        agent: Target agent identifier for query routing
        reasoning: Step-by-step explanation of routing decision
        confidence: Quantitative confidence score (0.0-1.0)
    """
    agent: str  # Target agent for query processing
    reasoning: str  # Transparent decision-making rationale
    confidence: float  # Statistical confidence in routing decision


class AgentConfig:
    """
    Comprehensive configuration settings for medical agent orchestration system.
    
    This class centralizes all system parameters, model selections, and routing logic
    to ensure consistent behavior across the multi-agent medical workflow.
    Clinical decision-making thresholds are based on evidence-based medicine principles.
    """
    
    # Model configuration for decision-making and analysis
    DECISION_MODEL = "gpt-4o"  # Primary LLM for agent routing decisions
    VISION_MODEL = "gpt-4o"   # Vision model for medical image classification
    
    # Clinical decision-making thresholds
    CONFIDENCE_THRESHOLD = 0.85  # Minimum confidence for automated routing
    
    # Comprehensive system prompt for intelligent medical triage
    DECISION_SYSTEM_PROMPT = """You are an intelligent medical triage system that routes user queries to 
    the appropriate specialized agent. Your job is to analyze the user's request and determine which agent 
    is best suited to handle it based on the query content, presence of images, and conversation context.

    Available agents:
    1. CONVERSATION_AGENT - For general chat, greetings, and non-medical questions.
    2. RAG_AGENT - For specific medical knowledge questions that can be answered from established medical literature. 
       Currently ingested medical knowledge involves 'introduction to brain tumor', 'deep learning techniques to 
       diagnose and detect brain tumors', 'deep learning techniques to diagnose and detect covid / covid-19 from chest x-ray'.
    3. WEB_SEARCH_PROCESSOR_AGENT - For questions about recent medical developments, current outbreaks, or time-sensitive medical information.
    4. BRAIN_TUMOR_AGENT - For analysis of brain MRI images to detect and segment tumors.
    5. CHEST_XRAY_AGENT - For analysis of chest X-ray images to detect abnormalities.
    6. SKIN_LESION_AGENT - For analysis of skin lesion images to classify them as benign or malignant.

    Make your decision based on these guidelines:
    - If the user has not uploaded any image, always route to the conversation agent.
    - If the user uploads a medical image, decide which medical vision agent is appropriate based on the image type and the user's query. 
      If the image is uploaded without a query, always route to the correct medical vision agent based on the image type.
    - If the user asks about recent medical developments or current health situations, use the web search processor agent.
    - If the user asks specific medical knowledge questions, use the RAG agent.
    - For general conversation, greetings, or non-medical questions, use the conversation agent. 
      But if image is uploaded, always go to the medical vision agents first.

    You must provide your answer in JSON format with the following structure:
    {{
    "agent": "AGENT_NAME",
    "reasoning": "Your step-by-step reasoning for selecting this agent",
    "confidence": 0.95  // Value between 0.0 and 1.0 indicating your confidence in this decision
    }}
    """
    
    # Initialize image analysis capability for medical image routing
    @classmethod
    def get_image_analyzer(cls):
        """Get initialized image analysis agent for medical image classification."""
        if not hasattr(cls, '_image_analyzer'):
            cls._image_analyzer = ImageAnalysisAgent(config=config)
        return cls._image_analyzer

print("📋 Agent State and Configuration Classes Defined")
print(f"   🎯 Decision Model: {AgentConfig.DECISION_MODEL}")
print(f"   📊 Confidence Threshold: {AgentConfig.CONFIDENCE_THRESHOLD}")
print(f"   🔬 Available Agents: 6 specialized medical agents")
print(f"   🛡️ Safety Features: Guardrails and human validation integrated")
logger.info("Agent state and configuration classes successfully defined")

2025-08-11 14:57:25,658 - AgentOrchestration - INFO - Agent state and configuration classes successfully defined


📋 Agent State and Configuration Classes Defined
   🎯 Decision Model: gpt-4o
   📊 Confidence Threshold: 0.85
   🔬 Available Agents: 6 specialized medical agents
   🛡️ Safety Features: Guardrails and human validation integrated


## 3. LangGraph Workflow Construction

This section implements the core LangGraph workflow that orchestrates our multi-agent medical system. The architecture follows enterprise-grade workflow patterns:

### Workflow Architecture Components:

#### **3.1 Input Analysis Node**
- **Image Detection**: Automatic medical image type classification
- **Guardrails Check**: Input safety validation and content filtering
- **Context Preparation**: Conversation history and state initialization

#### **3.2 Intelligent Routing System**
- **Direct Medical Image Routing**: Bypass expensive LLM calls for classified images
- **OpenAI Decision Chain**: Advanced reasoning for complex query types
- **Confidence-Based Decisions**: Statistical validation of routing choices

#### **3.3 Agent Execution Nodes**
- **Parallel Processing**: Independent agent execution with state management
- **Error Handling**: Graceful failure recovery and fallback mechanisms
- **Performance Monitoring**: Execution time tracking and optimization

#### **3.4 Validation and Safety Pipeline**
- **Human Validation**: Medical diagnosis oversight requirements
- **Output Guardrails**: Response safety and appropriateness checking
- **Quality Assurance**: Confidence scoring and accuracy validation

### Clinical Decision-Making Flow:
The workflow implements evidence-based medical triage principles:
1. **Immediate Safety Check**: Input validation and content filtering
2. **Image Priority Routing**: Direct routing for classified medical images
3. **Knowledge Hierarchy**: Local → Web → Human validation escalation
4. **Quality Control**: Confidence thresholds and validation requirements

In [4]:
def create_medical_agent_graph():
    """
    Create and configure the comprehensive LangGraph workflow for medical agent orchestration.
    
    This function constructs a sophisticated state graph that manages the complete
    medical consultation workflow, from initial query analysis through specialized
    agent execution to final response validation and delivery.
    
    The workflow implements clinical triage principles with automated routing,
    safety guardrails, and human validation requirements for medical decisions.
    
    Returns:
        Compiled LangGraph workflow with memory persistence
    """
    
    logger.info("Initializing medical agent orchestration graph")
    
    # Initialize core system components
    memory = MemorySaver()  # Conversation persistence across sessions
    guardrails = LocalGuardrails(config.rag.llm)  # Safety and content validation
    decision_model = config.agent_decision.llm  # Primary decision-making LLM
    json_parser = JsonOutputParser(pydantic_object=AgentDecision)  # Structured output parsing
    
    # Create the agent decision prompt template
    decision_prompt = ChatPromptTemplate.from_messages([
        ("system", AgentConfig.DECISION_SYSTEM_PROMPT),
        ("human", "{input}")
    ])
    
    # Construct the decision-making chain
    decision_chain = decision_prompt | decision_model | json_parser
    
    print("🏗️ Building LangGraph Workflow Architecture...")
    print("   📊 Components: Decision chain, guardrails, memory persistence")
    print("   🔄 Workflow: Input → Analysis → Routing → Execution → Validation")
    
    # === WORKFLOW NODE DEFINITIONS ===
    
    def analyze_input(state: AgentState) -> AgentState:
        """
        Comprehensive input analysis including image detection and guardrails validation.
        
        This function serves as the entry point for all user interactions,
        performing essential preprocessing steps including medical image
        classification and content safety validation.
        
        Args:
            state: Current agent state with user input
            
        Returns:
            Updated state with analysis results and routing flags
        """
        logger.info("Starting input analysis phase")
        
        current_input = state["current_input"]
        has_image = False
        image_type = None
        
        # Extract text content for processing
        input_text = ""
        if isinstance(current_input, str):
            input_text = current_input
        elif isinstance(current_input, dict):
            input_text = current_input.get("text", "")
        
        # Apply input safety guardrails validation
        if input_text:
            is_allowed, guardrail_message = guardrails.check_input(input_text)
            if not is_allowed:
                logger.warning(f"Input blocked by guardrails: {input_text[:100]}...")
                print(f"🛡️ Input Safety Check: Content blocked by guardrails")
                return {
                    **state,
                    "messages": AIMessage(content=guardrail_message),
                    "agent_name": "INPUT_GUARDRAILS",
                    "has_image": False,
                    "image_type": None,
                    "bypass_routing": True  # Skip normal workflow
                }
        
        # Process medical image if present
        if isinstance(current_input, dict) and "image" in current_input:
            has_image = True
            image_path = current_input.get("image")
            
            logger.info(f"Processing medical image: {image_path}")
            print(f"📸 Medical Image Detected: Analyzing image type...")
            
            try:
                # Classify medical image type using GPT-4o Vision
                image_analyzer = AgentConfig.get_image_analyzer()
                image_analysis_result = image_analyzer.analyze_image(image_path)
                
                # Extract image type from analysis result
                if isinstance(image_analysis_result, dict):
                    image_type = image_analysis_result.get('image_type', 'UNKNOWN')
                elif isinstance(image_analysis_result, str):
                    # Parse string response for image type
                    image_type = "UNKNOWN"
                    if "chest x-ray" in image_analysis_result.lower():
                        image_type = "CHEST X-RAY"
                    elif "brain" in image_analysis_result.lower() and "mri" in image_analysis_result.lower():
                        image_type = "BRAIN MRI"
                    elif "skin" in image_analysis_result.lower() or "lesion" in image_analysis_result.lower():
                        image_type = "SKIN LESION"
                
                print(f"🔍 Image Classification Result: {image_type}")
                logger.info(f"Image classified as: {image_type}")
                
            except Exception as e:
                logger.error(f"Image analysis failed: {str(e)}")
                print(f"⚠️ Image Analysis Warning: {str(e)}")
                image_type = "UNKNOWN"
        
        return {
            **state,
            "has_image": has_image,
            "image_type": image_type,
            "bypass_routing": False  # Continue normal workflow
        }
    
    def check_routing_bypass(state: AgentState) -> str:
        """
        Determine if normal agent routing should be bypassed due to guardrails.
        
        Args:
            state: Current agent state
            
        Returns:
            Routing decision: 'apply_guardrails' or 'route_to_agent'
        """
        if state.get("bypass_routing", False):
            return "apply_guardrails"
        return "route_to_agent"
    
    print("   ✅ Input Analysis Node: Image detection and guardrails validation")
    return decision_chain, analyze_input, check_routing_bypass

# Initialize the workflow construction
decision_chain, analyze_input, check_routing_bypass = create_medical_agent_graph()
logger.info("Medical agent graph components successfully initialized")

2025-08-11 14:57:25,668 - AgentOrchestration - INFO - Initializing medical agent orchestration graph
2025-08-11 14:57:25,669 - AgentOrchestration - INFO - Medical agent graph components successfully initialized


🏗️ Building LangGraph Workflow Architecture...
   📊 Components: Decision chain, guardrails, memory persistence
   🔄 Workflow: Input → Analysis → Routing → Execution → Validation
   ✅ Input Analysis Node: Image detection and guardrails validation


In [5]:
# Continue building the LangGraph workflow with intelligent routing system

def create_routing_system():
    """
    Create the sophisticated routing system that determines optimal agent selection.
    
    This system implements a two-tier routing approach:
    1. Direct routing for classified medical images (efficiency optimization)
    2. OpenAI decision chain for complex queries (reasoning optimization)
    
    Returns:
        Routing function for agent selection
    """
    
    def route_to_agent(state: AgentState) -> Dict:
        """
        Intelligent agent routing based on input analysis and clinical triage principles.
        
        This function implements a sophisticated decision tree that routes queries
        to the most appropriate agent based on content analysis, image presence,
        and conversation context. The routing logic prioritizes efficiency and
        clinical accuracy while maintaining cost-effective operation.
        
        Args:
            state: Current agent state with input and analysis results
            
        Returns:
            Dictionary with updated state and routing decision
        """
        logger.info("Starting intelligent agent routing")
        
        messages = state["messages"]
        current_input = state["current_input"]
        has_image = state["has_image"]
        image_type = state["image_type"]
        
        # === DIRECT ROUTING FOR MEDICAL IMAGES ===
        # This optimization bypasses expensive LLM calls for classified medical images
        if has_image and image_type:
            image_type_upper = image_type.upper()
            
            # Direct routing to specialized medical vision agents
            if "CHEST X-RAY" in image_type_upper or "CHEST_XRAY" in image_type_upper:
                logger.info("Direct routing: Chest X-ray → CHEST_XRAY_AGENT")
                print(f"🫁 Direct Routing: Chest X-ray detected → CHEST_XRAY_AGENT")
                updated_state = {**state, "agent_name": "CHEST_XRAY_AGENT"}
                return {"agent_state": updated_state, "next": "CHEST_XRAY_AGENT"}
            
            elif "BRAIN MRI" in image_type_upper or "BRAIN_TUMOR" in image_type_upper:
                logger.info("Direct routing: Brain MRI → BRAIN_TUMOR_AGENT")
                print(f"🧠 Direct Routing: Brain MRI detected → BRAIN_TUMOR_AGENT")
                updated_state = {**state, "agent_name": "BRAIN_TUMOR_AGENT"}
                return {"agent_state": updated_state, "next": "BRAIN_TUMOR_AGENT"}
            
            elif "SKIN LESION" in image_type_upper:
                logger.info("Direct routing: Skin lesion → SKIN_LESION_AGENT")
                print(f"👤 Direct Routing: Skin lesion detected → SKIN_LESION_AGENT")
                updated_state = {**state, "agent_name": "SKIN_LESION_AGENT"}
                return {"agent_state": updated_state, "next": "SKIN_LESION_AGENT"}
            
            elif "NON-MEDICAL" in image_type_upper or "OTHER" in image_type_upper:
                logger.info("Direct routing: Non-medical image → CONVERSATION_AGENT")
                print(f"💬 Direct Routing: Non-medical image → CONVERSATION_AGENT")
                updated_state = {**state, "agent_name": "CONVERSATION_AGENT"}
                return {"agent_state": updated_state, "next": "CONVERSATION_AGENT"}
        
        # === OPENAI DECISION CHAIN FOR COMPLEX QUERIES ===
        # For non-image queries or unclassified images, use advanced reasoning
        logger.info("Using OpenAI decision chain for complex query routing")
        print(f"🤖 Advanced Routing: Using OpenAI decision chain for query analysis")
        
        # Prepare input text for decision model
        input_text = ""
        if isinstance(current_input, str):
            input_text = current_input
        elif isinstance(current_input, dict):
            input_text = current_input.get("text", "")
        
        # Create conversation context from recent message history
        recent_context = ""
        for msg in messages[-6:]:  # Last 3 exchanges (6 messages)
            if isinstance(msg, HumanMessage):
                recent_context += f"User: {msg.content}\n"
            elif isinstance(msg, AIMessage):
                recent_context += f"Assistant: {msg.content}\n"
        
        # Construct comprehensive decision input
        decision_input = f"""
        User query: {input_text}

        Recent conversation context:
        {recent_context}

        Has image: {has_image}
        Image type: {image_type if has_image else 'None'}

        Based on this information, which agent should handle this query?
        """
        
        try:
            # Execute decision chain with comprehensive input
            decision = decision_chain.invoke({"input": decision_input})
            
            agent_name = decision["agent"]
            confidence = decision.get("confidence", 0.0)
            reasoning = decision.get("reasoning", "No reasoning provided")
            
            logger.info(f"Decision made: {agent_name} (confidence: {confidence:.2f})")
            print(f"🎯 Routing Decision: {agent_name}")
            print(f"📊 Confidence Score: {confidence:.2f}")
            print(f"💭 Reasoning: {reasoning[:100]}{'...' if len(reasoning) > 100 else ''}")
            
            # Update state with decision results
            updated_state = {**state, "agent_name": agent_name}
            
            # Route based on confidence threshold
            if confidence < AgentConfig.CONFIDENCE_THRESHOLD:
                logger.warning(f"Low confidence routing: {confidence:.2f} < {AgentConfig.CONFIDENCE_THRESHOLD}")
                print(f"⚠️ Low Confidence: Routing to validation workflow")
                return {"agent_state": updated_state, "next": "needs_validation"}
            
            return {"agent_state": updated_state, "next": agent_name}
            
        except Exception as e:
            logger.error(f"Decision chain failed: {str(e)}")
            print(f"❌ Routing Error: Falling back to CONVERSATION_AGENT")
            # Fallback to conversation agent on decision failure
            updated_state = {**state, "agent_name": "CONVERSATION_AGENT"}
            return {"agent_state": updated_state, "next": "CONVERSATION_AGENT"}
    
    return route_to_agent

# Initialize the routing system
route_to_agent = create_routing_system()
print("   ✅ Intelligent Routing System: Direct image routing + OpenAI decision chain")
logger.info("Intelligent routing system successfully initialized")

2025-08-11 14:57:25,678 - AgentOrchestration - INFO - Intelligent routing system successfully initialized


   ✅ Intelligent Routing System: Direct image routing + OpenAI decision chain


In [6]:
# ========== LANGGRAPH WORKFLOW CONSTRUCTION ==========
# This section constructs the complete multi-agent medical workflow using LangGraph
# It integrates all agents, routing logic, validation, and guardrails into a cohesive system

def create_medical_agent_workflow(all_agents):
    """
    Creates the complete LangGraph workflow for medical agent orchestration.
    This workflow integrates all 6 specialized agents with intelligent routing, 
    confidence-based decisions, and human validation requirements.
    
    Returns:
        StateGraph: Compiled LangGraph workflow ready for medical consultations
    """
    print("🏗️ Constructing Medical Agent Orchestration Workflow...")
    
    # Import LangGraph components
    from langgraph.graph import StateGraph, END
    from langgraph.checkpoint.memory import MemorySaver
    
    # Initialize workflow memory for conversation persistence
    memory = MemorySaver()
    workflow = StateGraph(AgentState)
    
    # ========== WORKFLOW NODE DEFINITIONS ==========
    
    def analyze_input_node(state):
        """Entry point: Analyzes user input and applies initial guardrails"""
        print("🔍 STEP 1: Analyzing user input and applying guardrails...")
        
        # Extract user input
        current_input = state.get("current_input", "")
        input_text = current_input if isinstance(current_input, str) else current_input.get("text", "")
        
        # Apply input guardrails
        is_safe, filtered_input, warning = validation_system['apply_input_guardrails'](input_text)
        
        if not is_safe:
            return {
                **state,
                "output": warning,
                "agent_name": "INPUT_GUARDRAILS",
                "bypass_routing": True
            }
        
        # Check for medical images
        has_image = isinstance(current_input, dict) and "image_path" in current_input
        image_type = None
        
        if has_image:
            print("📸 Medical image detected - Analyzing image type...")
            # Simulate image classification (in real implementation, use GPT-4o Vision)
            image_path = current_input.get("image_path", "")
            if "brain" in image_path.lower() or "mri" in image_path.lower():
                image_type = "BRAIN MRI"
            elif "chest" in image_path.lower() or "xray" in image_path.lower():
                image_type = "CHEST X-RAY"
            elif "skin" in image_path.lower() or "lesion" in image_path.lower():
                image_type = "SKIN LESION"
            else:
                image_type = "OTHER"
        
        return {
            **state,
            "has_image": has_image,
            "image_type": image_type,
            "bypass_routing": False
        }
    
    def intelligent_routing_node(state):
        """Smart routing: Direct image routing or OpenAI-powered decision making"""
        print("🎯 STEP 2: Intelligent agent routing...")
        
        has_image = state.get("has_image", False)
        image_type = state.get("image_type", "")
        
        # DIRECT ROUTING FOR MEDICAL IMAGES (Cost optimization + Speed)
        if has_image and image_type:
            print(f"🚀 Direct routing for medical image: {image_type}")
            
            if "BRAIN MRI" in image_type.upper():
                return {**state, "agent_name": "BRAIN_TUMOR_AGENT", "next_agent": "BRAIN_TUMOR_AGENT"}
            elif "CHEST X-RAY" in image_type.upper():
                return {**state, "agent_name": "CHEST_XRAY_AGENT", "next_agent": "CHEST_XRAY_AGENT"}
            elif "SKIN LESION" in image_type.upper():
                return {**state, "agent_name": "SKIN_LESION_AGENT", "next_agent": "SKIN_LESION_AGENT"}
            else:
                return {**state, "agent_name": "CONVERSATION_AGENT", "next_agent": "CONVERSATION_AGENT"}
        
        # OPENAI DECISION CHAIN FOR TEXT QUERIES
        print("🤖 Using OpenAI decision chain for text query routing...")
        current_input = state.get("current_input", "")
        input_text = current_input if isinstance(current_input, str) else current_input.get("text", "")
        
        # Simple rule-based routing for demo (in real implementation, use OpenAI)
        input_lower = input_text.lower()
        
        if any(word in input_lower for word in ['recent', 'latest', 'current', 'news', 'outbreak']):
            agent_decision = "WEB_SEARCH_PROCESSOR_AGENT"
        elif any(word in input_lower for word in ['brain tumor', 'covid', 'medical literature', 'research']):
            agent_decision = "RAG_AGENT"
        else:
            agent_decision = "CONVERSATION_AGENT"
        
        print(f"📋 Routing decision: {agent_decision}")
        return {**state, "agent_name": agent_decision, "next_agent": agent_decision}
    
    # ========== WORKFLOW EDGE FUNCTIONS ==========
    
    def check_bypass_routing(state):
        """Determines if normal routing should be bypassed due to guardrails"""
        return "apply_final_guardrails" if state.get("bypass_routing", False) else "intelligent_routing"
    
    def route_to_next_agent(state):
        """Routes to the selected agent based on routing decision"""
        return state.get("next_agent", "CONVERSATION_AGENT")
    
    def check_rag_confidence(state):
        """Checks RAG confidence and routes to web search if needed"""
        return validation_system['route_based_on_confidence'](state)
    
    def check_human_validation_needed(state):
        """Determines if human validation is required"""
        needs_validation = state.get("needs_human_validation", False)
        return "human_validation" if needs_validation else "apply_final_guardrails"
    
    # ========== ADD WORKFLOW NODES ==========
    
    print("📋 Adding workflow nodes...")
    
    # Core workflow nodes
    workflow.add_node("analyze_input", analyze_input_node)
    workflow.add_node("intelligent_routing", intelligent_routing_node)
    
    # Agent execution nodes (using our previously created agents)
    workflow.add_node("CONVERSATION_AGENT", all_agents["CONVERSATION_AGENT"])
    workflow.add_node("RAG_AGENT", all_agents["RAG_AGENT"])
    workflow.add_node("WEB_SEARCH_PROCESSOR_AGENT", all_agents["WEB_SEARCH_PROCESSOR_AGENT"])
    workflow.add_node("BRAIN_TUMOR_AGENT", all_agents["BRAIN_TUMOR_AGENT"])
    workflow.add_node("CHEST_XRAY_AGENT", all_agents["CHEST_XRAY_AGENT"])
    workflow.add_node("SKIN_LESION_AGENT", all_agents["SKIN_LESION_AGENT"])
    
    # Validation and guardrails nodes
    workflow.add_node("check_validation", lambda state: {**state, "needs_human_validation": state.get("needs_human_validation", False)})
    workflow.add_node("human_validation", lambda state: {
        **state, 
        "output": validation_system['create_human_validation_prompt'](
            state.get("output", ""), 
            state.get("agent_name", "")
        )
    })
    workflow.add_node("apply_final_guardrails", lambda state: {
    **state,
    "output": validation_system['apply_output_guardrails'](
        state["output"], state["current_input"]
    )
})
    
    # ========== DEFINE WORKFLOW EDGES ==========
    
    print("🔗 Defining workflow connections...")
    
    # Set entry point
    workflow.set_entry_point("analyze_input")
    
    # Input analysis to routing (with bypass check)
    workflow.add_conditional_edges(
        "analyze_input",
        check_bypass_routing,
        {
            "intelligent_routing": "intelligent_routing",
            "apply_final_guardrails": "apply_final_guardrails"
        }
    )
    
    # Routing to specific agents
    workflow.add_conditional_edges(
        "intelligent_routing",
        route_to_next_agent,
        {
            "CONVERSATION_AGENT": "CONVERSATION_AGENT",
            "RAG_AGENT": "RAG_AGENT", 
            "WEB_SEARCH_PROCESSOR_AGENT": "WEB_SEARCH_PROCESSOR_AGENT",
            "BRAIN_TUMOR_AGENT": "BRAIN_TUMOR_AGENT",
            "CHEST_XRAY_AGENT": "CHEST_XRAY_AGENT",
            "SKIN_LESION_AGENT": "SKIN_LESION_AGENT"
        }
    )
    
    # Agent outputs to validation checks
    workflow.add_edge("CONVERSATION_AGENT", "check_validation")
    workflow.add_edge("WEB_SEARCH_PROCESSOR_AGENT", "check_validation")
    workflow.add_edge("BRAIN_TUMOR_AGENT", "check_validation")
    workflow.add_edge("CHEST_XRAY_AGENT", "check_validation")
    workflow.add_edge("SKIN_LESION_AGENT", "check_validation")
    
    # RAG agent with confidence-based routing
    workflow.add_conditional_edges(
        "RAG_AGENT",
        check_rag_confidence,
        {
            "WEB_SEARCH_PROCESSOR_AGENT": "WEB_SEARCH_PROCESSOR_AGENT",
            "check_validation": "check_validation"
        }
    )
    
    # Validation check to human validation or final guardrails
    workflow.add_conditional_edges(
        "check_validation",
        check_human_validation_needed,
        {
            "human_validation": "human_validation",
            "apply_final_guardrails": "apply_final_guardrails"
        }
    )
    
    # Final steps
    workflow.add_edge("human_validation", "apply_final_guardrails")
    workflow.add_edge("apply_final_guardrails", END)
    
    # ========== COMPILE WORKFLOW ==========
    
    print("⚙️ Compiling LangGraph workflow...")
    compiled_workflow = workflow.compile(checkpointer=memory)
    
    print("✅ Medical Agent Orchestration Workflow successfully created!")
    print("🔧 Workflow includes:")
    print("   - 6 Specialized Medical Agents")
    print("   - Intelligent Routing System") 
    print("   - Confidence-Based Decision Making")
    print("   - Human Validation Workflows")
    print("   - Input/Output Guardrails")
    print("   - Conversation Memory Persistence")
    
    return compiled_workflow

# Create the complete medical workflow
# medical_workflow = create_medical_agent_workflow(all_agents)  # Will execute after agents are combined


## 4. Specialized Agent Implementations

This section implements all six specialized medical agents that form the core of our multi-agent system. Each agent is designed with specific medical expertise and integrates seamlessly with the LangGraph orchestration framework.

### Agent Architecture Principles:

#### **4.1 Clinical Specialization**
- **Domain Expertise**: Each agent focuses on specific medical domains
- **Evidence-Based Responses**: Grounded in established medical knowledge
- **Safety Protocols**: Built-in validation and oversight requirements

#### **4.2 Technical Integration**
- **Consistent Interfaces**: Standardized input/output formats across agents
- **Error Handling**: Graceful failure recovery and user feedback
- **Performance Monitoring**: Execution time tracking and optimization

#### **4.3 Quality Assurance**
- **Confidence Scoring**: Quantitative reliability assessment
- **Source Attribution**: Transparent reference to medical literature
- **Human Validation**: Medical diagnosis oversight requirements

### Medical Agent Catalog:

1. **CONVERSATION_AGENT**: General medical consultation and patient interaction
2. **RAG_AGENT**: Evidence-based responses from medical literature
3. **WEB_SEARCH_PROCESSOR_AGENT**: Current medical information retrieval
4. **BRAIN_TUMOR_AGENT**: YOLO-based MRI tumor detection
5. **CHEST_XRAY_AGENT**: DenseNet COVID-19 chest X-ray analysis
6. **SKIN_LESION_AGENT**: U-Net skin lesion segmentation

In [7]:
def create_specialized_agents():
    """
    Create all six specialized medical agents with comprehensive functionality.
    
    Each agent is designed for specific medical tasks while maintaining
    consistent interfaces and quality standards across the system.
    
    Returns:
        Dictionary containing all agent functions
    """
    
    def run_conversation_agent(state: AgentState) -> AgentState:
        """
        General medical conversation agent for patient interaction and consultation.
        
        This agent handles general medical questions, patient education,
        and conversational interactions while maintaining clinical accuracy
        and professional standards.
        
        Args:
            state: Current agent state with conversation context
            
        Returns:
            Updated state with conversation response
        """
        logger.info("Executing CONVERSATION_AGENT")
        print(f"💬 Selected Agent: CONVERSATION_AGENT")
        
        messages = state["messages"]
        current_input = state["current_input"]
        
        # Extract input text for processing
        input_text = ""
        if isinstance(current_input, str):
            input_text = current_input
        elif isinstance(current_input, dict):
            input_text = current_input.get("text", "")
        
        # Build conversation context from message history
        recent_context = ""
        for msg in messages:  # Include complete conversation history
            if isinstance(msg, HumanMessage):
                recent_context += f"User: {msg.content}\n"
            elif isinstance(msg, AIMessage):
                recent_context += f"Assistant: {msg.content}\n"
        
        # Construct comprehensive conversation prompt
        conversation_prompt = f"""User query: {input_text}

Recent conversation context: {recent_context}

You are an AI-powered Medical Conversation Assistant. Your goal is to facilitate smooth and informative 
conversations with users, handling both casual and medical-related queries. You must respond naturally 
while ensuring medical accuracy and clarity.

### Role & Capabilities
- Engage in **general conversation** while maintaining professionalism.
- Answer **medical questions** using verified knowledge.
- Route **complex queries** to RAG (retrieval-augmented generation) or web search if needed.
- Handle **follow-up questions** while keeping track of conversation context.
- Redirect **medical images** to the appropriate AI analysis agent.

### Guidelines for Responding:
1. **General Conversations:**
   - If the user engages in casual talk (e.g., greetings, small talk), respond in a friendly, engaging manner.
   - Keep responses **concise and engaging**, unless a detailed answer is needed.

2. **Medical Questions:**
   - If you have **high confidence** in answering, provide a medically accurate response.
   - Ensure responses are **clear, concise, and factual**.

3. **Follow-Up & Clarifications:**
   - Maintain conversation history for better responses.
   - If a query is unclear, ask **follow-up questions** before answering.

4. **Handling Medical Image Analysis:**
   - Do **not** attempt to analyze images yourself.
   - If user speaks about analyzing or processing or detecting or segmenting or classifying any disease 
     from any image, ask the user to upload the image so that in the next turn it is routed to the 
     appropriate medical vision agents.
   - If an image was uploaded, it would have been routed to the medical computer vision agents. 
     Read the history to know about the diagnosis results and continue conversation if user asks 
     anything regarding the diagnosis.
   - After processing, **help the user interpret the results**.

5. **Uncertainty & Ethical Considerations:**
   - If unsure, **never assume** medical facts.
   - Recommend consulting a **licensed healthcare professional** for serious medical concerns.
   - Avoid providing **medical diagnoses** or **prescriptions**—stick to general knowledge.

### Response Format:
- Maintain a **conversational yet professional tone**.
- Use **bullet points or numbered lists** for clarity when needed.
- If pulling from external sources (RAG/Web Search), mention **where the information is from** 
  (e.g., "According to Mayo Clinic...").
- If a user asks for a diagnosis, remind them to **seek medical consultation**.

Conversational LLM Response:"""
        
        try:
            # Generate response using conversation LLM
            start_time = time.time()
            response = config.conversation.llm.invoke(conversation_prompt)
            processing_time = time.time() - start_time
            
            logger.info(f"Conversation response generated in {processing_time:.2f}s")
            print(f"   ⏱️ Processing Time: {processing_time:.2f} seconds")
            print(f"   📝 Response Length: {len(response.content)} characters")
            
            return {
                **state,
                "output": response,
                "agent_name": "CONVERSATION_AGENT"
            }
            
        except Exception as e:
            logger.error(f"Conversation agent error: {str(e)}")
            error_response = AIMessage(content="I apologize, but I'm experiencing technical difficulties. Please try your question again.")
            return {
                **state,
                "output": error_response,
                "agent_name": "CONVERSATION_AGENT"
            }
    
    def run_rag_agent(state: AgentState) -> AgentState:
        """
        Medical knowledge retrieval agent using RAG (Retrieval-Augmented Generation).
        
        This agent searches through pre-ingested medical literature to provide
        evidence-based responses with source attribution and confidence scoring.
        
        Args:
            state: Current agent state with query context
            
        Returns:
            Updated state with RAG response and confidence metrics
        """
        logger.info("Executing RAG_AGENT")
        print(f"📚 Selected Agent: RAG_AGENT (Medical Literature Retrieval)")
        
        try:
            # Initialize RAG system
            rag_agent = MedicalRAG(config)
            
            messages = state["messages"]
            query = state["current_input"]
            rag_context_limit = config.rag.context_limit
            
            # Build conversation context for RAG query
            recent_context = ""
            for msg in messages[-rag_context_limit:]:
                if isinstance(msg, HumanMessage):
                    recent_context += f"User: {msg.content}\n"
                elif isinstance(msg, AIMessage):
                    recent_context += f"Assistant: {msg.content}\n"
            
            # Process query through RAG system
            start_time = time.time()
            response = rag_agent.process_query(query, chat_history=recent_context)
            processing_time = time.time() - start_time
            
            retrieval_confidence = response.get("confidence", 0.0)
            sources = response.get("sources", [])
            
            logger.info(f"RAG query processed in {processing_time:.2f}s with confidence {retrieval_confidence:.2f}")
            print(f"   ⏱️ Processing Time: {processing_time:.2f} seconds")
            print(f"   📊 Retrieval Confidence: {retrieval_confidence:.2f}")
            print(f"   📖 Sources Found: {len(sources)}")
            
            # Extract response content
            response_content = response["response"]
            if hasattr(response_content, 'content'):
                response_text = response_content.content
            else:
                response_text = str(response_content)
            
            # Check for insufficient information indicators
            insufficient_info = False
            insufficient_indicators = [
                "don't have enough information",
                "not enough information",
                "insufficient information",
                "cannot answer",
                "unable to answer"
            ]
            
            if isinstance(response_text, str):
                for indicator in insufficient_indicators:
                    if indicator in response_text.lower():
                        insufficient_info = True
                        logger.info("RAG response indicates insufficient information")
                        print(f"   ⚠️ Information Gap: Insufficient local knowledge detected")
                        break
            
            # Create response message
            if retrieval_confidence >= config.rag.min_retrieval_confidence:
                response_output = AIMessage(content=response_text)
            else:
                response_output = AIMessage(content="")
            
            return {
                **state,
                "output": response_output,
                "needs_human_validation": False,
                "retrieval_confidence": retrieval_confidence,
                "agent_name": "RAG_AGENT",
                "insufficient_info": insufficient_info
            }
            
        except Exception as e:
            logger.error(f"RAG agent error: {str(e)}")
            error_response = AIMessage(content="I'm having trouble accessing medical literature right now. Please try again or ask about current medical information.")
            return {
                **state,
                "output": error_response,
                "agent_name": "RAG_AGENT",
                "retrieval_confidence": 0.0,
                "insufficient_info": True
            }
    
    print("   ✅ Conversation Agent: General medical consultation and interaction")
    print("   ✅ RAG Agent: Medical literature retrieval with confidence scoring")
    
    return {
        "CONVERSATION_AGENT": run_conversation_agent,
        "RAG_AGENT": run_rag_agent
    }

# Initialize conversation and knowledge agents
text_agents = create_specialized_agents()
logger.info("Text-based medical agents successfully initialized")

2025-08-11 14:57:25,702 - AgentOrchestration - INFO - Text-based medical agents successfully initialized


   ✅ Conversation Agent: General medical consultation and interaction
   ✅ RAG Agent: Medical literature retrieval with confidence scoring


In [8]:
def create_web_search_and_vision_agents():
    """
    Create web search and medical vision agents for comprehensive medical analysis.
    
    Returns:
        Dictionary containing web search and vision agent functions
    """
    
    def run_web_search_processor_agent(state: AgentState) -> AgentState:
        """
        Web search processor agent for current medical information retrieval.
        
        This agent handles queries about recent medical developments, current
        health situations, and time-sensitive medical information using web search.
        
        Args:
            state: Current agent state with query context
            
        Returns:
            Updated state with web search results
        """
        logger.info("Executing WEB_SEARCH_PROCESSOR_AGENT")
        print(f"🌐 Selected Agent: WEB_SEARCH_PROCESSOR_AGENT (Current Medical Info)")
        
        try:
            messages = state["messages"]
            web_search_context_limit = config.web_search.context_limit
            
            # Build conversation context for web search
            recent_context = ""
            for msg in messages[-web_search_context_limit:]:
                if isinstance(msg, HumanMessage):
                    recent_context += f"User: {msg.content}\n"
                elif isinstance(msg, AIMessage):
                    recent_context += f"Assistant: {msg.content}\n"
            
            # Initialize web search processor
            web_search_processor = WebSearchProcessorAgent(config)
            
            # Process query with web search
            start_time = time.time()
            processed_response = web_search_processor.process_web_search_results(
                query=state["current_input"], 
                chat_history=recent_context
            )
            processing_time = time.time() - start_time
            
            logger.info(f"Web search completed in {processing_time:.2f}s")
            print(f"   ⏱️ Processing Time: {processing_time:.2f} seconds")
            
            # FIX: Extract content from AIMessage object before using len()
            if hasattr(processed_response, 'content'):
                response_content = processed_response.content
                response_length = len(response_content)
            else:
                response_content = str(processed_response)
                response_length = len(response_content)
                
            print(f"   📝 Response Length: {response_length} characters")
            
            # Determine involved agents for attribution
            if state['agent_name']:
                involved_agents = f"{state['agent_name']}, WEB_SEARCH_PROCESSOR_AGENT"
            else:
                involved_agents = "WEB_SEARCH_PROCESSOR_AGENT"
            
            return {
                **state,
                "output": AIMessage(content=response_content),  # ✅ FIXED
                "agent_name": involved_agents
            }
            
        except Exception as e:
            logger.error(f"Web search agent error: {str(e)}")
            error_response = AIMessage(content="I'm having trouble accessing current medical information. Please try again or ask about established medical knowledge.")
            return {
                **state,
                "output": error_response,
                "agent_name": "WEB_SEARCH_PROCESSOR_AGENT"
            }
    
    def run_brain_tumor_agent(state: AgentState) -> AgentState:
        """
        Brain tumor detection agent using YOLO model for MRI analysis.
        
        This agent processes brain MRI images to detect and localize tumors
        using state-of-the-art YOLO object detection with medical visualization.
        
        Args:
            state: Current agent state with image input
            
        Returns:
            Updated state with tumor detection results
        """
        logger.info("Executing BRAIN_TUMOR_AGENT")
        print(f"🧠 Selected Agent: BRAIN_TUMOR_AGENT (YOLO Tumor Detection)")
        
        try:
            current_input = state["current_input"]
            image_path = None

            if isinstance(current_input, dict):
                # Try both possible key names
                image_path = current_input.get("image") or current_input.get("image_path")
            elif isinstance(current_input, str):
                # If current_input is a string, it might be the image path directly
                image_path = current_input
            
            if not image_path:
                raise ValueError("No image path provided for brain tumor analysis")
            
            print(f"   📸 Analyzing MRI image: {os.path.basename(image_path)}")
            
            # Initialize image analyzer
            image_analyzer = AgentConfig.get_image_analyzer()
            
            # Perform tumor detection
            start_time = time.time()
            predicted_class = image_analyzer.classify_brain_tumor(image_path)
            detection_time = time.time() - start_time
            
            # Generate visualization
            visualization_success = image_analyzer.visualize_brain_tumor_detection(image_path)
            
            logger.info(f"Brain tumor analysis completed in {detection_time:.2f}s: {predicted_class}")
            print(f"   ⏱️ Analysis Time: {detection_time:.2f} seconds")
            print(f"   🎯 Detection Result: {predicted_class}")
            print(f"   📊 Visualization: {'✅ Generated' if visualization_success else '❌ Failed'}")
            
            # Generate clinical response
            if predicted_class == "tumor_detected":
                response = AIMessage(content="**BRAIN TUMOR ANALYSIS RESULTS**\n\n" +
                                   "🚨 **TUMOR DETECTED**: The YOLO analysis of the uploaded brain MRI image indicates the presence of a potential brain tumor. " +
                                   "The AI system has identified suspicious regions that require immediate medical attention.\n\n" +
                                   "**⚠️ IMPORTANT**: This AI analysis is for preliminary screening only and must be validated by a qualified radiologist or neurologist. " +
                                   "Please consult with a healthcare professional immediately for proper diagnosis and treatment planning.")
            elif predicted_class == "no_tumor":
                response = AIMessage(content="**BRAIN TUMOR ANALYSIS RESULTS**\n\n" +
                                   "✅ **NO TUMOR DETECTED**: The YOLO analysis of the uploaded brain MRI image indicates no obvious tumor structures. " +
                                   "The scan appears to show normal brain tissue patterns.\n\n" +
                                   "**📋 NOTE**: While no tumor was detected by the AI system, this does not replace professional medical evaluation. " +
                                   "Clinical correlation with symptoms and professional radiological interpretation is recommended.")
            else:
                response = AIMessage(content="**BRAIN TUMOR ANALYSIS RESULTS**\n\n" +
                                   "⚠️ **ANALYSIS INCONCLUSIVE**: The uploaded image could not be properly analyzed by the AI system. " +
                                   "This may be due to image quality, format issues, or the image not being a brain MRI scan.\n\n" +
                                   "**📋 RECOMMENDATION**: Please ensure the image is a clear brain MRI scan and try again, or consult with a medical professional.")
            
            return {
                **state,
                "output": response,
                "needs_human_validation": True,  # All medical diagnoses require validation
                "agent_name": "BRAIN_TUMOR_AGENT"
            }
            
        except Exception as e:
            logger.error(f"Brain tumor agent error: {str(e)}")
            error_response = AIMessage(content="I encountered an error while analyzing the brain MRI image. Please ensure the image is a clear MRI scan and try again.")
            return {
                **state,
                "output": error_response,
                "agent_name": "BRAIN_TUMOR_AGENT"
            }
    
    print("   ✅ Web Search Agent: Current medical information retrieval")
    print("   ✅ Brain Tumor Agent: YOLO-based MRI tumor detection")
    
    return {
        "WEB_SEARCH_PROCESSOR_AGENT": run_web_search_processor_agent,
        "BRAIN_TUMOR_AGENT": run_brain_tumor_agent
    }

# Initialize web search and brain tumor agents
web_and_brain_agents = create_web_search_and_vision_agents()
logger.info("Web search and brain tumor agents successfully initialized")

2025-08-11 14:57:25,712 - AgentOrchestration - INFO - Web search and brain tumor agents successfully initialized


   ✅ Web Search Agent: Current medical information retrieval
   ✅ Brain Tumor Agent: YOLO-based MRI tumor detection


## 5. Completing Medical Vision Agents

This section completes our medical vision agent implementations with the chest X-ray and skin lesion analysis capabilities. These agents represent the pinnacle of AI-assisted medical imaging analysis.

### Advanced Medical Vision Capabilities:

#### **5.1 Chest X-ray COVID-19 Detection**
- **DenseNet-121 Architecture**: State-of-the-art convolutional neural network
- **COVID-19 Pattern Recognition**: Trained on validated radiological datasets
- **Clinical Correlation**: Results aligned with radiological findings
- **Quality Assurance**: Confidence scoring and validation requirements

#### **5.2 Skin Lesion Segmentation**
- **U-Net Architecture**: Gold standard for medical image segmentation
- **Pixel-Level Precision**: Exact lesion boundary delineation
- **Dermatological Applications**: ABCDE rule assessment support
- **Visualization**: Advanced overlay plotting for clinical interpretation

### Clinical Safety Framework:
All medical vision agents implement comprehensive safety protocols:
- **Human Validation Requirements**: Medical diagnoses flagged for oversight
- **Confidence Thresholds**: Statistical reliability assessment
- **Error Handling**: Graceful failure recovery and user guidance
- **Ethical Guidelines**: Appropriate disclaimers and professional referrals

In [9]:
def create_remaining_vision_agents():
    """
    Create the remaining medical vision agents for comprehensive image analysis.
    
    Returns:
        Dictionary containing chest X-ray and skin lesion agent functions
    """
    
    def run_chest_xray_agent(state: AgentState) -> AgentState:
        """
        Chest X-ray analysis agent using DenseNet for COVID-19 detection.
        
        This agent processes chest X-ray images to detect COVID-19 patterns
        using a DenseNet-121 convolutional neural network trained on validated
        radiological datasets with confirmed diagnoses.
        
        Args:
            state: Current agent state with image input
            
        Returns:
            Updated state with COVID-19 detection results
        """
        logger.info("Executing CHEST_XRAY_AGENT")
        print(f"🫁 Selected Agent: CHEST_XRAY_AGENT (DenseNet COVID-19 Detection)")
        
        try:
            current_input = state["current_input"]
            image_path = None

            if isinstance(current_input, dict):
                image_path = current_input.get("image") or current_input.get("image_path")
            elif isinstance(current_input, str):
                image_path = current_input
            
            if not image_path:
                raise ValueError("No image path provided for chest X-ray analysis")
            
            print(f"   📸 Analyzing chest X-ray: {os.path.basename(image_path)}")
            
            # Initialize image analyzer
            image_analyzer = AgentConfig.get_image_analyzer()
            
            # Perform COVID-19 classification
            start_time = time.time()
            predicted_class = image_analyzer.classify_chest_xray(image_path)
            classification_time = time.time() - start_time
            
            logger.info(f"Chest X-ray analysis completed in {classification_time:.2f}s: {predicted_class}")
            print(f"   ⏱️ Analysis Time: {classification_time:.2f} seconds")
            print(f"   🎯 Classification Result: {predicted_class}")
            
            # Generate clinical response based on classification
            if "covid19" in str(predicted_class).lower():
                response = AIMessage(content="**CHEST X-RAY ANALYSIS RESULTS**\n\n" +
                                   "🚨 **COVID-19 POSITIVE**: The DenseNet analysis of the uploaded chest X-ray image indicates patterns consistent with COVID-19 pneumonia. " +
                                   "The AI system has detected radiological features commonly associated with COVID-19 infection.\n\n" +
                                   "**📋 CLINICAL RECOMMENDATIONS:**\n" +
                                   "• Immediate isolation protocols should be initiated\n" +
                                   "• RT-PCR confirmation testing is strongly recommended\n" +
                                   "• Clinical correlation with symptoms and exposure history\n" +
                                   "• Contact tracing procedures as per local guidelines\n\n" +
                                   "**⚠️ IMPORTANT**: This AI analysis is for preliminary screening only and must be validated by a qualified radiologist. " +
                                   "Please consult with a healthcare professional immediately for proper diagnosis and treatment planning.")
            elif "normal" in str(predicted_class).lower():
                response = AIMessage(content="**CHEST X-RAY ANALYSIS RESULTS**\n\n" +
                                   "✅ **COVID-19 NEGATIVE**: The DenseNet analysis of the uploaded chest X-ray image indicates normal lung patterns with no obvious signs of COVID-19 pneumonia. " +
                                   "The radiological features appear consistent with healthy lung tissue.\n\n" +
                                   "**📋 CLINICAL NOTES:**\n" +
                                   "• No obvious COVID-19 radiological signs detected\n" +
                                   "• Continue standard clinical assessment if symptomatic\n" +
                                   "• Consider alternative diagnoses if respiratory symptoms persist\n" +
                                   "• Follow local health guidelines for testing if indicated\n\n" +
                                   "**📋 NOTE**: While no COVID-19 patterns were detected, this does not rule out infection entirely. " +
                                   "Clinical correlation and professional medical evaluation are recommended, especially if symptoms are present.")
            else:
                response = AIMessage(content="**CHEST X-RAY ANALYSIS RESULTS**\n\n" +
                                   "⚠️ **ANALYSIS INCONCLUSIVE**: The uploaded image could not be properly classified by the AI system. " +
                                   "This may be due to image quality issues, incorrect format, or the image not being a chest X-ray.\n\n" +
                                   "**📋 RECOMMENDATIONS:**\n" +
                                   "• Ensure the image is a clear chest X-ray (PA or AP view)\n" +
                                   "• Check image quality and format\n" +
                                   "• Consider re-uploading with better image quality\n" +
                                   "• Consult with a medical professional for proper evaluation")
            
            return {
                **state,
                "output": response,
                "needs_human_validation": True,  # All medical diagnoses require validation
                "agent_name": "CHEST_XRAY_AGENT"
            }
            
        except Exception as e:
            logger.error(f"Chest X-ray agent error: {str(e)}")
            error_response = AIMessage(content="I encountered an error while analyzing the chest X-ray image. Please ensure the image is a clear chest X-ray and try again.")
            return {
                **state,
                "output": error_response,
                "agent_name": "CHEST_XRAY_AGENT"
            }
    
    def run_skin_lesion_agent(state: AgentState) -> AgentState:
        """
        Skin lesion segmentation agent using U-Net for dermatological analysis.
        
        This agent processes dermoscopic images to perform precise pixel-level
        segmentation of skin lesions using U-Net architecture optimized for
        medical image analysis.
        
        Args:
            state: Current agent state with image input
            
        Returns:
            Updated state with skin lesion segmentation results
        """
        logger.info("Executing SKIN_LESION_AGENT")
        print(f"👤 Selected Agent: SKIN_LESION_AGENT (U-Net Lesion Segmentation)")
        
        try:
            current_input = state["current_input"]
            image_path = None

            if isinstance(current_input, dict):
                image_path = current_input.get("image") or current_input.get("image_path")
            elif isinstance(current_input, str):
                image_path = current_input
            
            if not image_path:
                raise ValueError("No image path provided for skin lesion analysis")
            
            print(f"   📸 Analyzing skin lesion: {os.path.basename(image_path)}")
            
            # Initialize image analyzer
            image_analyzer = AgentConfig.get_image_analyzer()
            
            # Perform lesion segmentation
            start_time = time.time()
            segmentation_result = image_analyzer.segment_skin_lesion(image_path)
            segmentation_time = time.time() - start_time
            
            logger.info(f"Skin lesion segmentation completed in {segmentation_time:.2f}s: {segmentation_result}")
            print(f"   ⏱️ Analysis Time: {segmentation_time:.2f} seconds")
            print(f"   🎯 Segmentation Result: {segmentation_result}")
            
            # Check segmentation success
            segmentation_success = False
            if isinstance(segmentation_result, bool):
                segmentation_success = segmentation_result
            elif isinstance(segmentation_result, str):
                segmentation_success = any(indicator in segmentation_result.lower() 
                                         for indicator in ['success', 'completed', 'true'])
            
            # Generate clinical response based on segmentation results
            if segmentation_success:
                response = AIMessage(content="**SKIN LESION ANALYSIS RESULTS**\n\n" +
                                   "✅ **SEGMENTATION COMPLETED**: The U-Net analysis has successfully segmented the uploaded skin lesion image with pixel-level precision. " +
                                   "The AI system has identified and delineated the lesion boundaries for detailed morphological analysis.\n\n" +
                                   "**📊 DERMATOLOGICAL APPLICATIONS:**\n" +
                                   "• **Lesion Area Measurement**: Precise quantification for tracking changes over time\n" +
                                   "• **Border Irregularity Assessment**: Analysis of lesion edge characteristics\n" +
                                   "• **Asymmetry Analysis**: Evaluation of lesion symmetry patterns\n" +
                                   "• **Color Distribution Mapping**: Assessment of pigmentation variations\n" +
                                   "• **Follow-up Comparison**: Baseline for future monitoring studies\n\n" +
                                   "**🔍 ABCDE RULE SUPPORT:**\n" +
                                   "The segmentation provides quantitative data to support evaluation of:\n" +
                                   "• **A**symmetry: Morphological analysis of lesion shape\n" +
                                   "• **B**order: Edge irregularity and definition assessment\n" +
                                   "• **C**olor: Pigmentation distribution mapping\n" +
                                   "• **D**iameter: Precise area and perimeter measurements\n" +
                                   "• **E**volution: Baseline for monitoring changes\n\n" +
                                   "**⚠️ CLINICAL NOTE**: This AI segmentation is a diagnostic aid and requires validation by a qualified dermatologist. " +
                                   "Please consult with a healthcare professional for proper diagnosis and treatment recommendations.")
            else:
                response = AIMessage(content="**SKIN LESION ANALYSIS RESULTS**\n\n" +
                                   "⚠️ **SEGMENTATION INCOMPLETE**: The uploaded image could not be properly segmented by the AI system. " +
                                   "This may be due to image quality issues, incorrect format, or the image not being a clear skin lesion.\n\n" +
                                   "**📋 RECOMMENDATIONS:**\n" +
                                   "• Ensure the image is a clear dermoscopic or clinical photo of a skin lesion\n" +
                                   "• Check image quality, lighting, and focus\n" +
                                   "• Avoid images with excessive hair, artifacts, or poor contrast\n" +
                                   "• Consider re-uploading with better image quality\n" +
                                   "• Consult with a dermatologist for professional evaluation\n\n" +
                                   "**🏥 ALTERNATIVE**: If you have concerns about a skin lesion, please schedule an appointment with a dermatologist for proper clinical examination.")
            
            return {
                **state,
                "output": response,
                "needs_human_validation": True,  # All medical diagnoses require validation
                "agent_name": "SKIN_LESION_AGENT"
            }
            
        except Exception as e:
            logger.error(f"Skin lesion agent error: {str(e)}")
            error_response = AIMessage(content="I encountered an error while analyzing the skin lesion image. Please ensure the image is a clear dermatological photo and try again.")
            return {
                **state,
                "output": error_response,
                "agent_name": "SKIN_LESION_AGENT"
            }
    
    print("   ✅ Chest X-ray Agent: DenseNet COVID-19 detection with clinical recommendations")
    print("   ✅ Skin Lesion Agent: U-Net segmentation with dermatological analysis")
    
    return {
        "CHEST_XRAY_AGENT": run_chest_xray_agent,
        "SKIN_LESION_AGENT": run_skin_lesion_agent
    }

# Initialize remaining vision agents
vision_agents = create_remaining_vision_agents()
logger.info("All medical vision agents successfully initialized")
print("\n🎯 All Six Specialized Medical Agents Successfully Created:")
print("   💬 CONVERSATION_AGENT - General medical consultation")
print("   📚 RAG_AGENT - Medical literature retrieval")
print("   🌐 WEB_SEARCH_PROCESSOR_AGENT - Current medical information")
print("   🧠 BRAIN_TUMOR_AGENT - YOLO MRI tumor detection")
print("   🫁 CHEST_XRAY_AGENT - DenseNet COVID-19 detection")
print("   👤 SKIN_LESION_AGENT - U-Net lesion segmentation")

2025-08-11 14:57:25,725 - AgentOrchestration - INFO - All medical vision agents successfully initialized


   ✅ Chest X-ray Agent: DenseNet COVID-19 detection with clinical recommendations
   ✅ Skin Lesion Agent: U-Net segmentation with dermatological analysis

🎯 All Six Specialized Medical Agents Successfully Created:
   💬 CONVERSATION_AGENT - General medical consultation
   📚 RAG_AGENT - Medical literature retrieval
   🌐 WEB_SEARCH_PROCESSOR_AGENT - Current medical information
   🧠 BRAIN_TUMOR_AGENT - YOLO MRI tumor detection
   🫁 CHEST_XRAY_AGENT - DenseNet COVID-19 detection
   👤 SKIN_LESION_AGENT - U-Net lesion segmentation


In [10]:
# Combine all agent functions into a single dictionary for workflow integration
all_agents = {
    **text_agents,
    **web_and_brain_agents, 
    **vision_agents
}

print("🔄 Agent Integration Status:")
for agent_name in all_agents.keys():
    print(f"   ✅ {agent_name.replace('run_', '').replace('_', ' ').title()}")

logger.info(f"All {len(all_agents)} medical agents successfully integrated")

# Create the complete medical workflow now that all agents are defined
print("🏗️ Creating Medical Agent Workflow...")
medical_workflow = create_medical_agent_workflow(all_agents)
print("✅ Medical workflow ready for consultations\!")


2025-08-11 14:57:25,729 - AgentOrchestration - INFO - All 6 medical agents successfully integrated


🔄 Agent Integration Status:
   ✅ Conversation Agent
   ✅ Rag Agent
   ✅ Web Search Processor Agent
   ✅ Brain Tumor Agent
   ✅ Chest Xray Agent
   ✅ Skin Lesion Agent
🏗️ Creating Medical Agent Workflow...
🏗️ Constructing Medical Agent Orchestration Workflow...
📋 Adding workflow nodes...
🔗 Defining workflow connections...
⚙️ Compiling LangGraph workflow...
✅ Medical Agent Orchestration Workflow successfully created!
🔧 Workflow includes:
   - 6 Specialized Medical Agents
   - Intelligent Routing System
   - Confidence-Based Decision Making
   - Human Validation Workflows
   - Input/Output Guardrails
   - Conversation Memory Persistence
✅ Medical workflow ready for consultations\!


In [11]:
# ========== GUARDRAILS AND VALIDATION SYSTEM ==========
# This section implements the safety and validation components that ensure medical outputs are safe and validated by humans when necessary

def create_validation_and_guardrails_system():
    """
    Creates the validation and guardrails system for medical AI responses.
    This system ensures safety, ethical compliance, and human oversight for medical diagnoses.
    
    Returns:
        dict: Dictionary containing validation and guardrails functions
    """
    print("🛡️ Initializing Medical AI Safety and Validation System...")
    
    def check_medical_confidence(state):
        """
        Evaluates the confidence of medical predictions and determines if human validation is needed.
        
        Args:
            state (dict): Current agent state with prediction confidence scores
            
        Returns:
            dict: Updated state with human validation requirements
        """
        confidence_threshold = 0.85  # High confidence threshold for medical decisions
        
        # Extract confidence from different agent outputs
        confidence_score = state.get('retrieval_confidence', 0.0)
        agent_name = state.get('agent_name', '')
        
        # Medical image analysis agents always require human validation
        medical_agents = ['BRAIN_TUMOR_AGENT', 'CHEST_XRAY_AGENT', 'SKIN_LESION_AGENT']
        
        if any(agent in agent_name for agent in medical_agents):
            print(f"🏥 Medical diagnosis detected from {agent_name} - Human validation REQUIRED")
            state['needs_human_validation'] = True
        elif confidence_score < confidence_threshold:
            print(f"⚠️ Low confidence score ({confidence_score:.2f}) - Human validation recommended")
            state['needs_human_validation'] = True
        else:
            print(f"✅ High confidence score ({confidence_score:.2f}) - No additional validation needed")
            state['needs_human_validation'] = False
            
        return state
    
    def apply_input_guardrails(user_input):
        """
        Applies safety filters to user input to prevent harmful or inappropriate queries.
        
        Args:
            user_input (str): Raw user input text
            
        Returns:
            tuple: (is_safe, filtered_input, warning_message)
        """
        print("🔍 Applying input safety guardrails...")
        
        # Define prohibited content patterns
        prohibited_patterns = [
            'suicide', 'self-harm', 'kill myself', 'end my life',
            'illegal drugs', 'prescription fraud', 'fake prescription',
            'medical advice for others', 'diagnose someone else'
        ]
        
        # Check for prohibited content
        input_lower = user_input.lower()
        for pattern in prohibited_patterns:
            if pattern in input_lower:
                warning_msg = f"⚠️ Input contains potentially harmful content: '{pattern}'. Please rephrase your query."
                print(f"🚫 Blocked input: Contains '{pattern}'")
                return False, "", warning_msg
        
        # Input passes safety checks
        print("✅ Input passes safety checks")
        return True, user_input, ""
    
    def apply_output_guardrails(generated_response, original_query):
        """
        Applies safety filters and ethical guidelines to AI-generated medical responses.
        
        Args:
            generated_response (str): AI-generated medical response
            original_query (str): Original user query for context
            
        Returns:
            str: Sanitized and ethically compliant response
        """
        print("🛡️ Applying output safety and ethical guardrails...")

        # FIX: Extract content from AIMessage object if needed
        if hasattr(generated_response, 'content'):
            response_content = generated_response.content
        else:
            response_content = str(generated_response)
        
        # Ensure medical disclaimers are present
        medical_keywords = ['diagnosis', 'treatment', 'medication', 'symptoms', 'disease', 'condition']
        
        if any(keyword in response_content.lower() for keyword in medical_keywords): 
            disclaimer = "\n\n⚕️ **Medical Disclaimer**: This AI analysis is for informational purposes only and should not replace professional medical advice. Always consult with a qualified healthcare provider for medical decisions."
            
            if disclaimer not in response_content:
                response_content += disclaimer
                print("📋 Added medical disclaimer to response")
        
        # Remove overly confident diagnostic language
        confidence_phrases = [
            ("you definitely have", "you may have"),
            ("this is certainly", "this could be"),
            ("you should immediately", "consider consulting a healthcare provider about"),
            ("this proves", "this suggests")
        ]
        
        for problematic, replacement in confidence_phrases:
            if problematic in response_content.lower():
                response_content = response_content.replace(problematic, replacement)
                print(f"🔄 Replaced overly confident language: '{problematic}' → '{replacement}'")

        from langchain_core.messages import AIMessage
        return AIMessage(content=response_content)
        
        #print("✅ Output passes ethical and safety guidelines")
        #return response_content
    
    def create_human_validation_prompt(medical_output, agent_name):
        """
        Creates a human validation prompt for medical professionals.
        
        Args:
            medical_output (str): AI-generated medical analysis
            agent_name (str): Name of the agent that generated the output
            
        Returns:
            str: Formatted validation prompt for human review
        """
        print(f"👨‍⚕️ Creating human validation prompt for {agent_name}")
        
        validation_prompt = f"""
🏥 **MEDICAL AI VALIDATION REQUIRED**

**AI Agent**: {agent_name}
**Analysis Output**: {medical_output}

**For Healthcare Professionals**:
Please review this AI-generated medical analysis and validate its accuracy:
- ✅ **APPROVE**: If the analysis appears medically sound and appropriate
- ❌ **REJECT**: If you identify any medical inaccuracies or concerns
- 💬 **COMMENTS**: Provide any additional clinical insights or corrections

**For Patients**:
This analysis has been generated by AI and requires validation by a healthcare professional.
Please seek consultation with a qualified medical provider for proper diagnosis and treatment.

**Validation Status**: ⏳ PENDING HUMAN REVIEW
        """
        
        return validation_prompt
    
    def route_based_on_confidence(state):
        """
        Implements intelligent routing based on confidence scores and information sufficiency.
        
        Args:
            state (dict): Current workflow state
            
        Returns:
            str: Next agent to route to ('WEB_SEARCH_PROCESSOR_AGENT' or 'END')
        """
        confidence = state.get('retrieval_confidence', 0.0)
        insufficient_info = state.get('insufficient_info', False)
        min_confidence = 0.75  # Minimum confidence threshold
        
        print(f"🎯 Routing decision - Confidence: {confidence:.2f}, Insufficient info: {insufficient_info}")
        
        if confidence < min_confidence or insufficient_info:
            print("🔄 Low confidence/insufficient info → Routing to Web Search Agent")
            return "WEB_SEARCH_PROCESSOR_AGENT"
        else:
            print("✅ High confidence → Proceeding to validation check")
            return "check_validation"
    
    # Return all validation and guardrails functions
    return {
        'check_medical_confidence': check_medical_confidence,
        'apply_input_guardrails': apply_input_guardrails,
        'apply_output_guardrails': apply_output_guardrails,
        'create_human_validation_prompt': create_human_validation_prompt,
        'route_based_on_confidence': route_based_on_confidence
    }

# Initialize the validation and guardrails system
validation_system = create_validation_and_guardrails_system()

print("✅ Validation and Guardrails System successfully initialized!")
print("📋 Available functions:")
for func_name in validation_system.keys():
    print(f"   - {func_name}")

🛡️ Initializing Medical AI Safety and Validation System...
✅ Validation and Guardrails System successfully initialized!
📋 Available functions:
   - check_medical_confidence
   - apply_input_guardrails
   - apply_output_guardrails
   - create_human_validation_prompt
   - route_based_on_confidence


# ========== INDIVIDUAL AGENT TESTING ==========

## Individual Agent Testing Suite

This section provides separate test cells for each medical agent, allowing you to:
- **Test agents individually** without running the full 19-test suite
- **Debug specific agent functionality** in isolation
- **Quickly verify agent responses** during development
- **Understand each agent's capabilities** and output format

### Available Individual Tests:

1. **🤖 Conversation Agent** - General medical consultation and patient interaction
2. **📚 RAG Agent** - Medical literature retrieval from knowledge base
3. **🌐 Web Search Agent** - Current medical information and recent developments
4. **🧠 Brain Tumor Agent** - MRI brain tumor detection using YOLO
5. **🫁 Chest X-ray Agent** - COVID-19 detection using DenseNet
6. **🩺 Skin Lesion Agent** - Skin lesion segmentation using U-Net

### Usage Instructions:

- **Run cells individually** by clicking the run button on each test cell
- **Modify test queries** in each cell to test different scenarios
- **Check agent outputs** and response times for each component
- **Use for debugging** when specific agents aren't working as expected

---


In [12]:
# ========== CONVERSATION AGENT INDIVIDUAL TEST ==========
print("🤖 TESTING CONVERSATION AGENT")
print("=" * 50)

# Test query - modify this to test different conversation scenarios
test_query = "Hello\! I'm experiencing mild headaches recently. What could be causing them?"

print(f"📝 Test Query: {test_query}")
print(f"🎯 Expected Agent: CONVERSATION_AGENT")
print("🚀 Executing workflow...")

# Create initial state
initial_state = {
    "current_input": test_query,
    "current_query": test_query,
    "has_image": False,
    "image_type": None,
    "messages": [],
    "agent_name": None,
    "output": None,
    "needs_human_validation": False,
    "retrieval_confidence": 0.0,
    "bypass_routing": False,
    "insufficient_info": False
}

# Execute workflow
import time
start_time = time.time()

try:
    thread_config = {"configurable": {"thread_id": "conversation_test"}}
    result = medical_workflow.invoke(initial_state, thread_config)
    
    # Process results
    selected_agent = result.get("agent_name", "UNKNOWN")
    final_output = result.get("output", "No output generated")
    
    # Handle AIMessage objects
    if hasattr(final_output, 'content'):
        final_output_text = final_output.content
    else:
        final_output_text = str(final_output)
    
    processing_time = time.time() - start_time
    
    print(f"✅ CONVERSATION AGENT TEST COMPLETED")
    print(f"👤 Selected Agent: {selected_agent}")
    print(f"⏱️ Processing Time: {processing_time:.2f} seconds")
    print(f"📏 Response Length: {len(final_output_text)} characters")
    print(f"💬 Agent Response:")
    print("-" * 50)
    print(final_output_text)
    print("-" * 50)
    
    # Verify agent selection
    if "CONVERSATION_AGENT" in selected_agent:
        print("🎯 ✅ CORRECT AGENT SELECTED")
    else:
        print(f"🎯 ❌ INCORRECT AGENT: Expected CONVERSATION_AGENT, got {selected_agent}")
        
except Exception as e:
    print(f"❌ ERROR during conversation agent test:")
    print(f"Error: {str(e)}")
    print(f"Type: {type(e).__name__}")

2025-08-11 14:57:25,766 - AgentOrchestration - INFO - Executing WEB_SEARCH_PROCESSOR_AGENT


🤖 TESTING CONVERSATION AGENT
📝 Test Query: Hello\! I'm experiencing mild headaches recently. What could be causing them?
🎯 Expected Agent: CONVERSATION_AGENT
🚀 Executing workflow...
🔍 STEP 1: Analyzing user input and applying guardrails...
🔍 Applying input safety guardrails...
✅ Input passes safety checks
🎯 STEP 2: Intelligent agent routing...
🤖 Using OpenAI decision chain for text query routing...
📋 Routing decision: WEB_SEARCH_PROCESSOR_AGENT
🌐 Selected Agent: WEB_SEARCH_PROCESSOR_AGENT (Current Medical Info)


2025-08-11 14:57:26,669 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-11 14:57:38,745 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-11 14:57:38,748 - AgentOrchestration - INFO - Web search completed in 12.98s


   ⏱️ Processing Time: 12.98 seconds
   📝 Response Length: 994 characters
🛡️ Applying output safety and ethical guardrails...
📋 Added medical disclaimer to response
✅ CONVERSATION AGENT TEST COMPLETED
👤 Selected Agent: WEB_SEARCH_PROCESSOR_AGENT, WEB_SEARCH_PROCESSOR_AGENT
⏱️ Processing Time: 12.99 seconds
📏 Response Length: 1201 characters
💬 Agent Response:
--------------------------------------------------
Mild headaches can be caused by a variety of factors. Hormonal changes, such as those experienced during pregnancy, can lead to headaches. Lifestyle factors like stress, depression, alcohol use, skipping meals, changes in sleep patterns, and taking too much medication can also trigger headaches. Environmental factors, including exposure to secondhand smoke, strong smells, allergens, certain foods, pollution, noise, lighting, and weather changes, can be potential triggers. Illnesses, including infections, colds, fevers, sinusitis, throat or ear infections, can cause headaches as wel

In [13]:
# ========== RAG AGENT INDIVIDUAL TEST ==========
print("📚 TESTING RAG AGENT")
print("=" * 50)

# Test query - modify this to test different knowledge retrieval scenarios
test_query = "What are the different types of brain tumors and their characteristics?"

print(f"📝 Test Query: {test_query}")
print(f"🎯 Expected Agent: RAG_AGENT")
print("🚀 Executing workflow...")

# Create initial state
initial_state = {
    "current_input": test_query,
    "current_query": test_query,
    "has_image": False,
    "image_type": None,
    "messages": [],
    "agent_name": None,
    "output": None,
    "needs_human_validation": False,
    "retrieval_confidence": 0.0,
    "bypass_routing": False,
    "insufficient_info": False
}

# Execute workflow
import time
start_time = time.time()

try:
    thread_config = {"configurable": {"thread_id": "rag_test"}}
    result = medical_workflow.invoke(initial_state, thread_config)
    
    # Process results
    selected_agent = result.get("agent_name", "UNKNOWN")
    final_output = result.get("output", "No output generated")
    retrieval_confidence = result.get("retrieval_confidence", 0.0)
    
    # Handle AIMessage objects
    if hasattr(final_output, 'content'):
        final_output_text = final_output.content
    else:
        final_output_text = str(final_output)
    
    processing_time = time.time() - start_time
    
    print(f"✅ RAG AGENT TEST COMPLETED")
    print(f"👤 Selected Agent: {selected_agent}")
    print(f"⏱️ Processing Time: {processing_time:.2f} seconds")
    print(f"📏 Response Length: {len(final_output_text)} characters")
    print(f"🔍 Retrieval Confidence: {retrieval_confidence:.3f}")
    print(f"📚 RAG Agent Response:")
    print("-" * 50)
    print(final_output_text)
    print("-" * 50)
    
    # Verify agent selection
    if "RAG_AGENT" in selected_agent:
        print("🎯 ✅ CORRECT AGENT SELECTED")
    else:
        print(f"🎯 ⚠️ AGENT: Expected RAG_AGENT, got {selected_agent}")
        print("   (May have been routed to Web Search due to low confidence)")
        
except Exception as e:
    print(f"❌ ERROR during RAG agent test:")
    print(f"   Error: {str(e)}")
    print(f"   Type: {type(e).__name__}")

2025-08-11 14:57:38,770 - AgentOrchestration - INFO - Executing RAG_AGENT
2025-08-11 14:57:38,771 - agents.rag_agent - INFO - Initializing Medical RAG system
2025-08-11 14:57:38,771 - agents.rag_agent.doc_parser - INFO - Medical Document Parser initialized!
2025-08-11 14:57:38,788 - AgentOrchestration - ERROR - RAG agent error: Storage folder ./data/qdrant_db is already accessed by another instance of Qdrant client. If you require concurrent access, use Qdrant server instead.
2025-08-11 14:57:38,789 - AgentOrchestration - INFO - Executing WEB_SEARCH_PROCESSOR_AGENT


📚 TESTING RAG AGENT
📝 Test Query: What are the different types of brain tumors and their characteristics?
🎯 Expected Agent: RAG_AGENT
🚀 Executing workflow...
🔍 STEP 1: Analyzing user input and applying guardrails...
🔍 Applying input safety guardrails...
✅ Input passes safety checks
🎯 STEP 2: Intelligent agent routing...
🤖 Using OpenAI decision chain for text query routing...
📋 Routing decision: RAG_AGENT
📚 Selected Agent: RAG_AGENT (Medical Literature Retrieval)
🎯 Routing decision - Confidence: 0.00, Insufficient info: True
🔄 Low confidence/insufficient info → Routing to Web Search Agent
🌐 Selected Agent: WEB_SEARCH_PROCESSOR_AGENT (Current Medical Info)


2025-08-11 14:57:39,561 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 503 Service Unavailable"
2025-08-11 14:57:39,563 - openai._base_client - INFO - Retrying request to /chat/completions in 0.468685 seconds
2025-08-11 14:57:41,229 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-11 14:57:53,281 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-11 14:57:53,285 - AgentOrchestration - INFO - Web search completed in 14.49s


   ⏱️ Processing Time: 14.49 seconds
   📝 Response Length: 1117 characters
🛡️ Applying output safety and ethical guardrails...
✅ RAG AGENT TEST COMPLETED
👤 Selected Agent: RAG_AGENT, WEB_SEARCH_PROCESSOR_AGENT
⏱️ Processing Time: 14.52 seconds
📏 Response Length: 1117 characters
🔍 Retrieval Confidence: 0.000
📚 RAG Agent Response:
--------------------------------------------------
Brain tumors are categorized into more than 150 types, based on their location, the type of cells they are made of, and whether they are cancerous (malignant) or noncancerous (benign). They can be primary, originating in the brain, or metastatic, spreading from cancer elsewhere in the body. 

Benign brain tumors are typically slow-growing, while malignant ones tend to grow fast. Some common benign brain tumors include meningioma and dysembryoplastic neuroepithelial tumor, which is usually found in children and teens and can cause seizures. 

Approximately 78% of malignant brain tumors are gliomas, which develop

In [15]:
# ========== WEB SEARCH PROCESSOR AGENT INDIVIDUAL TEST ==========
print("🌐 TESTING WEB SEARCH PROCESSOR AGENT")
print("=" * 50)

# Test query - modify this to test different web search scenarios
test_query = "What are the latest developments in COVID-19 treatment in 2025?"

print(f"📝 Test Query: {test_query}")
print(f"🎯 Expected Agent: WEB_SEARCH_PROCESSOR_AGENT")
print("🚀 Executing workflow...")

# Create initial state
initial_state = {
    "current_input": test_query,
    "current_query": test_query,
    "has_image": False,
    "image_type": None,
    "messages": [],
    "agent_name": None,
    "output": None,
    "needs_human_validation": False,
    "retrieval_confidence": 0.0,
    "bypass_routing": False,
    "insufficient_info": False
}

# Execute workflow
import time
start_time = time.time()

try:
    thread_config = {"configurable": {"thread_id": "websearch_test"}}
    result = medical_workflow.invoke(initial_state, thread_config)
    
    # Process results
    selected_agent = result.get("agent_name", "UNKNOWN")
    final_output = result.get("output", "No output generated")
    
    # Handle AIMessage objects
    if hasattr(final_output, 'content'):
        final_output_text = final_output.content
    else:
        final_output_text = str(final_output)
    
    processing_time = time.time() - start_time
    
    print(f"✅ WEB SEARCH PROCESSOR AGENT TEST COMPLETED")
    print(f"👤 Selected Agent: {selected_agent}")
    print(f"⏱️ Processing Time: {processing_time:.2f} seconds")
    print(f"📏 Response Length: {len(final_output_text)} characters")
    print(f"🌐 Web Search Response:")
    print("-" * 50)
    print(final_output_text)
    print("-" * 50)
    
    # Verify agent selection
    if "WEB_SEARCH_PROCESSOR_AGENT" in selected_agent:
        print("🎯 ✅ CORRECT AGENT SELECTED")
    else:
        print(f"🎯 ❌ INCORRECT AGENT: Expected WEB_SEARCH_PROCESSOR_AGENT, got {selected_agent}")
        
except Exception as e:
    print(f"❌ ERROR during web search agent test:")
    print(f"   Error: {str(e)}")
    print(f"   Type: {type(e).__name__}")
    print(f"   Note: Web search requires valid Tavily API key")

2025-08-11 14:58:38,928 - AgentOrchestration - INFO - Executing WEB_SEARCH_PROCESSOR_AGENT


🌐 TESTING WEB SEARCH PROCESSOR AGENT
📝 Test Query: What are the latest developments in COVID-19 treatment in 2025?
🎯 Expected Agent: WEB_SEARCH_PROCESSOR_AGENT
🚀 Executing workflow...
🔍 STEP 1: Analyzing user input and applying guardrails...
🔍 Applying input safety guardrails...
✅ Input passes safety checks
🎯 STEP 2: Intelligent agent routing...
🤖 Using OpenAI decision chain for text query routing...
📋 Routing decision: WEB_SEARCH_PROCESSOR_AGENT
🌐 Selected Agent: WEB_SEARCH_PROCESSOR_AGENT (Current Medical Info)


2025-08-11 14:58:40,182 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-11 14:58:56,821 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-11 14:58:56,825 - AgentOrchestration - INFO - Web search completed in 17.90s


   ⏱️ Processing Time: 17.90 seconds
   📝 Response Length: 1531 characters
🛡️ Applying output safety and ethical guardrails...
📋 Added medical disclaimer to response
✅ WEB SEARCH PROCESSOR AGENT TEST COMPLETED
👤 Selected Agent: WEB_SEARCH_PROCESSOR_AGENT, WEB_SEARCH_PROCESSOR_AGENT
⏱️ Processing Time: 17.91 seconds
📏 Response Length: 1738 characters
🌐 Web Search Response:
--------------------------------------------------
The latest developments in COVID-19 treatment in 2025 focus on understanding and treating the long-term effects of the virus, often referred to as Long COVID. The RECOVER Initiative is using data-driven research to study these effects, with recent studies focusing on cardiovascular and kidney function, as well as gastrointestinal outcomes in children and adolescents post-COVID.

In terms of treatment, ongoing trials are looking at the use of antivirals like valacyclovir, Truvada, and Maraviroc for Long COVID. Other potential treatments being tested include immunomodul

In [16]:
# ========== BRAIN TUMOR AGENT INDIVIDUAL TEST ==========
print("🧠 TESTING BRAIN TUMOR AGENT")
print("=" * 50)

# Test query with image - modify the image_path to test different brain MRI scans
test_query = {
    "text": "Please analyze this brain MRI scan for tumor detection",
    "image_path": "sample_images/brain_tumour/00059_141.jpg"
}

print(f"📝 Test Query: {test_query['text']}")
print(f"📸 Image Path: {test_query['image_path']}")
print(f"🎯 Expected Agent: BRAIN_TUMOR_AGENT")
print("🚀 Executing workflow...")

# Create initial state
initial_state = {
    "current_input": test_query,
    "current_query": test_query['text'],
    "has_image": True,
    "image_type": "BRAIN MRI",
    "uploaded_image_path": test_query['image_path'],
    "messages": [],
    "agent_name": None,
    "output": None,
    "needs_human_validation": False,
    "retrieval_confidence": 0.0,
    "bypass_routing": False,
    "insufficient_info": False
}

# Execute workflow
import time
start_time = time.time()

try:
    thread_config = {"configurable": {"thread_id": "brain_tumor_test"}}
    result = medical_workflow.invoke(initial_state, thread_config)
    
    # Process results
    selected_agent = result.get("agent_name", "UNKNOWN")
    final_output = result.get("output", "No output generated")
    needs_validation = result.get("needs_human_validation", False)
    
    # Handle AIMessage objects
    if hasattr(final_output, 'content'):
        final_output_text = final_output.content
    else:
        final_output_text = str(final_output)
    
    processing_time = time.time() - start_time
    
    print(f"✅ BRAIN TUMOR AGENT TEST COMPLETED")
    print(f"👤 Selected Agent: {selected_agent}")
    print(f"⏱️ Processing Time: {processing_time:.2f} seconds")
    print(f"📏 Response Length: {len(final_output_text)} characters")
    print(f"🏥 Needs Human Validation: {needs_validation}")
    print(f"🧠 Brain Tumor Analysis Result:")
    print("-" * 50)
    print(final_output_text)
    print("-" * 50)
    
    # Verify agent selection
    if "BRAIN_TUMOR_AGENT" in selected_agent:
        print("🎯 ✅ CORRECT AGENT SELECTED")
    else:
        print(f"🎯 ❌ INCORRECT AGENT: Expected BRAIN_TUMOR_AGENT, got {selected_agent}")
    
    # Check for visualization output
    print(f"📊 Check uploads/brain_tumor_output/ for visualization images")
        
except Exception as e:
    print(f"❌ ERROR during brain tumor agent test:")
    print(f"   Error: {str(e)}")
    print(f"   Type: {type(e).__name__}")
    print(f"   Note: Requires brain MRI image file and YOLO model")

2025-08-11 14:59:10,514 - AgentOrchestration - INFO - Executing BRAIN_TUMOR_AGENT
2025-08-11 14:59:10,514 - agents.image_analysis_agent.chest_xray_agent.covid_chest_xray_inference - INFO - Using device: cpu
2025-08-11 14:59:10,701 - agents.image_analysis_agent.chest_xray_agent.covid_chest_xray_inference - INFO - Model loaded successfully from ./agents/image_analysis_agent/chest_xray_agent/models/covid_chest_xray_model.pth


🧠 TESTING BRAIN TUMOR AGENT
📝 Test Query: Please analyze this brain MRI scan for tumor detection
📸 Image Path: sample_images/brain_tumour/00059_141.jpg
🎯 Expected Agent: BRAIN_TUMOR_AGENT
🚀 Executing workflow...
🔍 STEP 1: Analyzing user input and applying guardrails...
🔍 Applying input safety guardrails...
✅ Input passes safety checks
📸 Medical image detected - Analyzing image type...
🎯 STEP 2: Intelligent agent routing...
🚀 Direct routing for medical image: BRAIN MRI
🧠 Selected Agent: BRAIN_TUMOR_AGENT (YOLO Tumor Detection)
   📸 Analyzing MRI image: 00059_141.jpg


2025-08-11 14:59:10,762 - agents.image_analysis_agent.brain_tumor_agent.brain_tumor_inference - INFO - Brain tumor detection model loaded successfully from ./agents/image_analysis_agent/brain_tumor_agent/models/brain_tumour_od.pt
2025-08-11 14:59:10,873 - agents.image_analysis_agent.skin_lesion_agent.skin_lesion_inference - INFO - Model loaded successfully from ./agents/image_analysis_agent/skin_lesion_agent/models/checkpointN25_.pth.tar
2025-08-11 14:59:10,874 - agents.image_analysis_agent.brain_tumor_agent.brain_tumor_inference - ERROR - Error during brain tumor detection: Image file not found: sample_images/brain_tumour/00059_141.jpg
2025-08-11 14:59:10,874 - agents.image_analysis_agent.brain_tumor_agent.brain_tumor_inference - ERROR - Error generating brain tumor visualization: Image file not found: sample_images/brain_tumour/00059_141.jpg
2025-08-11 14:59:10,875 - AgentOrchestration - INFO - Brain tumor analysis completed in 0.00s: error


   ⏱️ Analysis Time: 0.00 seconds
   🎯 Detection Result: error
   📊 Visualization: ❌ Failed
👨‍⚕️ Creating human validation prompt for BRAIN_TUMOR_AGENT
🛡️ Applying output safety and ethical guardrails...
📋 Added medical disclaimer to response
✅ BRAIN TUMOR AGENT TEST COMPLETED
👤 Selected Agent: BRAIN_TUMOR_AGENT
⏱️ Processing Time: 0.37 seconds
📏 Response Length: 1294 characters
🏥 Needs Human Validation: True
🧠 Brain Tumor Analysis Result:
--------------------------------------------------

🏥 **MEDICAL AI VALIDATION REQUIRED**

**AI Agent**: BRAIN_TUMOR_AGENT
**Analysis Output**: content='**BRAIN TUMOR ANALYSIS RESULTS**\n\n⚠️ **ANALYSIS INCONCLUSIVE**: The uploaded image could not be properly analyzed by the AI system. This may be due to image quality, format issues, or the image not being a brain MRI scan.\n\n**📋 RECOMMENDATION**: Please ensure the image is a clear brain MRI scan and try again, or consult with a medical professional.' additional_kwargs={} response_metadata={}

**For 

In [17]:
# ========== CHEST X-RAY AGENT INDIVIDUAL TEST ==========
print("🫁 TESTING CHEST X-RAY AGENT")
print("=" * 50)

# Test query with image - modify the image_path to test different chest X-ray images
test_query = {
    "text": "Analyze this chest X-ray for COVID-19 signs",
    "image_path": "sample_images/chest_x-ray_covid_and_normal/NORMAL2-IM-0364-0001.jpeg"
}

print(f"📝 Test Query: {test_query['text']}")
print(f"📸 Image Path: {test_query['image_path']}")
print(f"🎯 Expected Agent: CHEST_XRAY_AGENT")
print("🚀 Executing workflow...")

# Create initial state
initial_state = {
    "current_input": test_query,
    "current_query": test_query['text'],
    "has_image": True,
    "image_type": "CHEST X-RAY",
    "uploaded_image_path": test_query['image_path'],
    "messages": [],
    "agent_name": None,
    "output": None,
    "needs_human_validation": False,
    "retrieval_confidence": 0.0,
    "bypass_routing": False,
    "insufficient_info": False
}

# Execute workflow
import time
start_time = time.time()

try:
    thread_config = {"configurable": {"thread_id": "chest_xray_test"}}
    result = medical_workflow.invoke(initial_state, thread_config)
    
    # Process results
    selected_agent = result.get("agent_name", "UNKNOWN")
    final_output = result.get("output", "No output generated")
    needs_validation = result.get("needs_human_validation", False)
    
    # Handle AIMessage objects
    if hasattr(final_output, 'content'):
        final_output_text = final_output.content
    else:
        final_output_text = str(final_output)
    
    processing_time = time.time() - start_time
    
    print(f"✅ CHEST X-RAY AGENT TEST COMPLETED")
    print(f"👤 Selected Agent: {selected_agent}")
    print(f"⏱️ Processing Time: {processing_time:.2f} seconds")
    print(f"📏 Response Length: {len(final_output_text)} characters")
    print(f"🏥 Needs Human Validation: {needs_validation}")
    print(f"🫁 Chest X-ray Analysis Result:")
    print("-" * 50)
    print(final_output_text)
    print("-" * 50)
    
    # Verify agent selection
    if "CHEST_XRAY_AGENT" in selected_agent:
        print("🎯 ✅ CORRECT AGENT SELECTED")
    else:
        print(f"🎯 ❌ INCORRECT AGENT: Expected CHEST_XRAY_AGENT, got {selected_agent}")
    
    # Show classification result
    if "COVID-19 POSITIVE" in final_output_text:
        print("🦠 COVID-19 POSITIVE detected")
    elif "COVID-19 NEGATIVE" in final_output_text:
        print("✅ COVID-19 NEGATIVE (Normal)")
        
except Exception as e:
    print(f"❌ ERROR during chest X-ray agent test:")
    print(f"   Error: {str(e)}")
    print(f"   Type: {type(e).__name__}")
    print(f"   Note: Requires chest X-ray image file and DenseNet model")

2025-08-11 14:59:20,952 - AgentOrchestration - INFO - Executing CHEST_XRAY_AGENT
2025-08-11 14:59:20,953 - agents.image_analysis_agent.chest_xray_agent.covid_chest_xray_inference - ERROR - Error during prediction Covid Chest X-ray: [Errno 2] No such file or directory: 'sample_images/chest_x-ray_covid_and_normal/NORMAL2-IM-0364-0001.jpeg'
2025-08-11 14:59:20,953 - AgentOrchestration - INFO - Chest X-ray analysis completed in 0.00s: None


🫁 TESTING CHEST X-RAY AGENT
📝 Test Query: Analyze this chest X-ray for COVID-19 signs
📸 Image Path: sample_images/chest_x-ray_covid_and_normal/NORMAL2-IM-0364-0001.jpeg
🎯 Expected Agent: CHEST_XRAY_AGENT
🚀 Executing workflow...
🔍 STEP 1: Analyzing user input and applying guardrails...
🔍 Applying input safety guardrails...
✅ Input passes safety checks
📸 Medical image detected - Analyzing image type...
🎯 STEP 2: Intelligent agent routing...
🚀 Direct routing for medical image: CHEST X-RAY
🫁 Selected Agent: CHEST_XRAY_AGENT (DenseNet COVID-19 Detection)
   📸 Analyzing chest X-ray: NORMAL2-IM-0364-0001.jpeg
   ⏱️ Analysis Time: 0.00 seconds
   🎯 Classification Result: None
👨‍⚕️ Creating human validation prompt for CHEST_XRAY_AGENT
🛡️ Applying output safety and ethical guardrails...
📋 Added medical disclaimer to response
✅ CHEST X-RAY AGENT TEST COMPLETED
👤 Selected Agent: CHEST_XRAY_AGENT
⏱️ Processing Time: 0.01 seconds
📏 Response Length: 1403 characters
🏥 Needs Human Validation: True
🫁 Ch

In [18]:
# ========== SKIN LESION AGENT INDIVIDUAL TEST ==========
print("🩺 TESTING SKIN LESION AGENT")
print("=" * 50)

# Test query with image - modify the image_path to test different skin lesion images
test_query = {
    "text": "Segment and analyze this skin lesion image",
    "image_path": "sample_images/skin_lesion_images/ISIC_0020849.jpg"
}

print(f"📝 Test Query: {test_query['text']}")
print(f"📸 Image Path: {test_query['image_path']}")
print(f"🎯 Expected Agent: SKIN_LESION_AGENT")
print("🚀 Executing workflow...")

# Create initial state
initial_state = {
    "current_input": test_query,
    "current_query": test_query['text'],
    "has_image": True,
    "image_type": "SKIN LESION",
    "uploaded_image_path": test_query['image_path'],
    "messages": [],
    "agent_name": None,
    "output": None,
    "needs_human_validation": False,
    "retrieval_confidence": 0.0,
    "bypass_routing": False,
    "insufficient_info": False
}

# Execute workflow
import time
start_time = time.time()

try:
    thread_config = {"configurable": {"thread_id": "skin_lesion_test"}}
    result = medical_workflow.invoke(initial_state, thread_config)
    
    # Process results
    selected_agent = result.get("agent_name", "UNKNOWN")
    final_output = result.get("output", "No output generated")
    needs_validation = result.get("needs_human_validation", False)
    
    # Handle AIMessage objects
    if hasattr(final_output, 'content'):
        final_output_text = final_output.content
    else:
        final_output_text = str(final_output)
    
    processing_time = time.time() - start_time
    
    print(f"✅ SKIN LESION AGENT TEST COMPLETED")
    print(f"👤 Selected Agent: {selected_agent}")
    print(f"⏱️ Processing Time: {processing_time:.2f} seconds")
    print(f"📏 Response Length: {len(final_output_text)} characters")
    print(f"🏥 Needs Human Validation: {needs_validation}")
    print(f"🩺 Skin Lesion Segmentation Result:")
    print("-" * 50)
    print(final_output_text)
    print("-" * 50)
    
    # Verify agent selection
    if "SKIN_LESION_AGENT" in selected_agent:
        print("🎯 ✅ CORRECT AGENT SELECTED")
    else:
        print(f"🎯 ❌ INCORRECT AGENT: Expected SKIN_LESION_AGENT, got {selected_agent}")
    
    # Check for segmentation output
    print(f"📊 Check uploads/skin_lesion_output/ for segmentation visualization")
        
except Exception as e:
    print(f"❌ ERROR during skin lesion agent test:")
    print(f"   Error: {str(e)}")
    print(f"   Type: {type(e).__name__}")
    print(f"   Note: Requires skin lesion image file and U-Net model")

2025-08-11 14:59:30,178 - AgentOrchestration - INFO - Executing SKIN_LESION_AGENT
[ WARN:0@126.575] global loadsave.cpp:268 findDecoder imread_('sample_images/skin_lesion_images/ISIC_0020849.jpg'): can't open/read file: check file path/integrity
2025-08-11 14:59:30,181 - agents.image_analysis_agent.skin_lesion_agent.skin_lesion_inference - ERROR - Error during segmentation: OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'

2025-08-11 14:59:30,182 - AgentOrchestration - ERROR - Skin lesion agent error: OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'



🩺 TESTING SKIN LESION AGENT
📝 Test Query: Segment and analyze this skin lesion image
📸 Image Path: sample_images/skin_lesion_images/ISIC_0020849.jpg
🎯 Expected Agent: SKIN_LESION_AGENT
🚀 Executing workflow...
🔍 STEP 1: Analyzing user input and applying guardrails...
🔍 Applying input safety guardrails...
✅ Input passes safety checks
📸 Medical image detected - Analyzing image type...
🎯 STEP 2: Intelligent agent routing...
🚀 Direct routing for medical image: SKIN LESION
👤 Selected Agent: SKIN_LESION_AGENT (U-Net Lesion Segmentation)
   📸 Analyzing skin lesion: ISIC_0020849.jpg
🛡️ Applying output safety and ethical guardrails...
✅ SKIN LESION AGENT TEST COMPLETED
👤 Selected Agent: SKIN_LESION_AGENT
⏱️ Processing Time: 0.01 seconds
📏 Response Length: 132 characters
🏥 Needs Human Validation: False
🩺 Skin Lesion Segmentation Result:
--------------------------------------------------
I encountered an error while analyzing the skin lesion image. Please ensure the image is a clear dermatological

## 🎯 Individual Agent Testing Complete\!

**You now have 6 separate test cells for individual agent testing:**

✅ **Conversation Agent Test** - General medical consultation
✅ **RAG Agent Test** - Medical knowledge retrieval
✅ **Web Search Agent Test** - Current medical information
✅ **Brain Tumor Agent Test** - MRI tumor detection
✅ **Chest X-ray Agent Test** - COVID-19 detection
✅ **Skin Lesion Agent Test** - Lesion segmentation

### Usage Tips:

- **Run individually**: Execute each test cell separately to test specific agents
- **Modify queries**: Change the test_query in each cell to test different scenarios
- **Check outputs**: Look for correct agent selection and response quality
- **Debug issues**: Use individual tests to isolate problems with specific agents
- **Performance monitoring**: Each test shows processing time and response metrics

### Expected Results:

- Each test should show "✅ CORRECT AGENT SELECTED"
- Response times should be reasonable (typically 2-15 seconds)
- Medical image agents should indicate "Needs Human Validation: True"
- Responses should be medically appropriate and well-formatted

**Happy Testing\! 🚀**